# **Setup**

Chúng tôi sẽ sử dụng 2 thư viện là underthesea và pyvi, các thư viện này được phát triển những năm gần đây, bao gồm Sentence segmentation, Tokenisation, Lowercasing, Stopword removal, Punctuation removal và Number replacement,...


In [2]:
# Download underthesea Open-source Vietnamese Natural Language Process Toolkit: https://github.com/undertheseanlp/underthesea
!pip install underthesea
# Download Python Vietnamese Toolkit: https://github.com/trungtv/pyvi 
!pip install pyvi

     |████████████████████████████████| 7.5 MB 4.3 MB/s 
     |████████████████████████████████| 2.9 MB 41.6 MB/s 
     |████████████████████████████████| 743 kB 53.6 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 235 kB 63.4 MB/s 
     |████████████████████████████████| 3.3 MB 57.2 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 636 kB 55.7 MB/s 
     |████████████████████████████████| 895 kB 60.9 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=6f291879334df74d78b7aa9537bcb3c0ca4eddd422fe4160b631f049296e5a45
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 8.5 MB 4

In [3]:
# Auto print ExecuteTime 
!pip install ipython-autotime
%load_ext autotime

time: 255 µs (started: 2021-10-09 03:29:25 +00:00)


# **Pre-processing**
Input: 
text1: Đoạn văn trích từ bài báo "Về xử lý tiếng Việt trong công nghệ thông tin" của Hồ Tú Bảo và Lương Chi Mai, gồm 7 câu, 523 từ và 2410 ký tự.

text2: 20 đoạn, 36 câu, 1344 từ, source: https://vnexpress.net/cuc-hang-khong-mo-cua-duong-bay-la-can-thiet-va-an-toan-4369038.html

text3: 130 đoạn, 354 câu, 16042 từ, 73334 ký tự, source: BTL DLDCSVN 

Output: danh sách các tokens

Quy trình demo pre-processing đi qua các bước sau:

1. Number replacement
2. Normalizing VNM Unicode 
3. Normalizing VNM accent marks placing 
4. Tokenization or just Sentence segmentation and end here!
5. Lowercasing
4. Removing stop words and punctuation

### **Prepare**

In [4]:
# Our selected paragraph. Complex enough to exemplify each step
text = """Hãy thử nhìn lại tình hình của chúng ta. Ngoài những việc đã làm và bước đầu làm được kể ở phần đầu, đã có những cố gắng trong nhiều nội dung khác nhau về xử lý ngôn ngữ và tiếng Việt. Trong các bài giảng về Trí tuệ Nhân tạo, về Lý thuyết Nhận dạng, về Xử lý Tín hiệu, về Khai phá Dữ liệu ở nhiều đại học, các nội dung và kỹ thuật xử lý ngôn ngữ đã ít nhiều được đề cập.Về xử lý tiếng nói và tiếng Việt, theo chúng tôi biết, hai tập thể làm nghiên cứu đã có những kết quả gần đây là Viện Công nghệ Thông tin và Trung tâm nghiên cứu quốc tế Thông tin đa phương tiện, truyền thông và ứng dụng (MICA) – Đại học Bách khoa Hà nội; một số kết quả ở một số trường Đại học là những đề tài tiến sĩ, thạc sĩ, mang tính chất tìm hiểu, chưa hệ thống và định hướng rõ ràng. Nghiên cứu về xử lý ngôn ngữ (văn bản) đã được theo đuổi bởi một số tập thể từ khá lâu (Đại học Bách khoa và Đại học Khoa học Tự nhiên thành phố Hồ Chí Minh, Đại học Bách Khoa Đà Nẵng, Đại học Bách khoa và Đại học Khoa học Tự nhiên Hà Nội, Trường Đại học Công nghệ, Viện Ứng dụng Công nghệ, Viện Công nghệ Thông tin, công ty Lạc Việt,…) về các vấn đề dịch máy, các bài toán cơ bản của xử lý tiếng Việt [Dien et al., 2001; Dien, 2003], [Huyen et al., 2003], và gần đây là tóm tắt văn bản [Minh, 2004; Minh et al., 2004], [Huong, 2004], tìm kiếm và trích chọn thông tin [Bao and Funakoshi, 1998], phân loại và chia nhóm văn bản [Bao and Binh, 2001], khai phá Web [Hieu, 2005], gióng hàng văn bản [Huyen et al., 2003], mô hình từ điển điện tử [Bao et al., 2003], xây dựng kho ngữ liệu [Dien, 2002], … và gần đây nhất là đề tài nhà nước “Nghiên cứu phát triển công nghệ nhận dạng, tổng hợp và xử lý ngôn ngữ tiếng Việt” giai đoạn 2001-2004 trong chương trình quốc gia KC-01 [Khang, 2004]. Bên ngoài Việt Nam, cũng có những nỗ lực về xử lý tiếng Việt, như nhóm dịch Anh-Việt của tiến sỹ Phạm Hải và bè bạn (Mỹ) khởi đầu từ đầu các năm 1990, của tiến sỹ Lê Tăng Hồ và phần mềm tổng hợp tiếng Việt VVV (Canada), … đặc biệt là của các cán bộ và nghiên cứu sinh Việt Nam tại Viện Khoa học và Công nghệ Tiên tiến Nhật bản (JAIST) với 6 nghiên cứu sinh về xử lý ngôn ngữ, hợp tác theo một kế hoạch thống nhất. Số nghiên cứu sinh về xử lý ngôn ngữ mới tốt 12 nghiệp như Lê Thanh Hương (Anh) [Huong, 2004], Nguyễn Thị Minh Huyền (Pháp), Hồ Bảo Quốc (Pháp), Nguyễn Lê Minh (Nhật) [Minh, 2004], và số sắp tốt nghiệp trong 1-2 năm tới đây ước tính khoảng 10 người."""

text1 = """Cục hàng không: Mở cửa đường bay là cần thiết và an toàn. Lãnh đạo Cục Hàng không Việt Nam đề nghị "các địa phương căn cứ vào tình hình thực tiễn để từng bước nối lại đường bay nội địa". Tại cuộc tọa đàm với các chuyên gia sáng 8/10, ông Võ Huy Cường - Cục phó Hàng không Việt Nam, nói việc mở cửa hàng không "đang là đòi hỏi bức thiết của xã hội".
"Đóng cửa rất dễ, nhưng việc này sẽ gây nhiều khó khăn cho cả địa phương, sân bay và kinh tế chung của đất nước", ông nói và cho rằng các địa phương cần chủ động đề ra biện pháp mở cửa trở lại, tạo hành lang an toàn để người dân, doanh nghiệp đi lại và tham gia các hoạt động kinh tế - xã hội.
Trong quá trình mở cửa, ngành hàng không "không thể làm thay các địa phương", mà chỉ có thể hỗ trợ phòng chống dịch bằng cách vận chuyển người có nhu cầu đi lại đáp ứng tiêu chuẩn kiểm soát Covid-19; sẵn sàng cung cấp thông tin lịch trình hành khách, dự báo số lượng từng ngày để các địa phương tính toán phương án...
Về việc Nội Bài là sân bay quốc gia nhưng phải chờ TP Hà Nội quyết định cho mở lại đường bay nội địa hay không, ông Cường nêu nhận định có thể chính quyền thủ đô quan điểm đây là một vùng đặc biệt quan trọng căn cứ theo Luật Thủ đô, khi có dịch bệnh hoặc vấn đề gì đó thì "đóng trước, mở sau". "Tình hình dịch bệnh trên toàn quốc là lý do Hà Nội quan ngại việc sẽ gánh chịu một làn sóng lây nhiễm, chúng tôi hoàn toàn hiểu và thông cảm với Hà Nội về điều đó", ông Cường nói.
Về phía Cục Hàng không Việt Nam, ông Cường nêu quan điểm "mở cửa lại đường hàng không là an toàn, từng bước, hiệu quả, bền vững". Hiện ngành hàng không có các cơ sở hậu cần kỹ thuật lớn tập trung ở sân bay Tân Sơn Nhất và Nội Bài.
Do vậy, nếu riêng Tân Sơn Nhất mở cửa, rõ ràng chỉ có một cơ sở hạ tầng kỹ thuật đáp ứng hiệu quả trong hoạt động khai thác. Các đường bay khác sẽ phải về Nội Bài, và nếu phải bay mà không có hành khách thì chắc chắn không hiệu quả, nghĩa là không thể bền vững.
Dẫn thống kê của Tổ chức Hàng không dân dụng quốc tế (ICAO), ông Võ Huy Cường cho hay đến cuối tháng 8/2020, có hơn một tỷ lượt khách quốc tế di chuyển bằng đường hàng không, chỉ phát hiện 41 hành khách lây nhiễm chéo. Thời gian qua, các hãng hàng không tổ chức bay nhiều chuyến vào vùng dịch đưa công dân về song không ai bị lây nhiễm.
"Ngành hàng không đã áp dụng tất cả các biện pháp phòng dịch nghiêm ngặt, cho thấy, việc di chuyển bằng đường hàng không đến giai đoạn hiện nay là an toàn nhất", ông Cường nhấn mạnh.
Theo ông, khi có sự cố xảy ra về lây nhiễm, hãng hàng không có thể nhanh chóng truy vết vì đã có đầy đủ thông tin hành khách là ai, đi chuyến bay nào, ngồi ghế nào, địa chỉ, nơi đến. Thực tế, ngành hàng không đã nhiều lần hỗ trợ cho Bộ Y tế truy vết, không chỉ quốc tế mà cả các chuyến bay nội địa khi nghi ngờ có F0 đi lại trên chuyến bay.
Theo Phó tổng giám đốc Tổng công ty Cảng hàng không Việt Nam (ACV) Nguyễn Quốc Phương, đơn vị đã họp với các hãng hàng không, đưa ra 4 tiêu chí: Hành khách xanh, quy trình phục vụ xanh, nhà ga - cơ sở hạ tầng xanh, máy bay - phương tiện vận chuyển xanh.
"Chúng tôi chuẩn bị rất chu đáo cho việc tái khôi phục hoạt động hàng không, sẽ đảm bảo an toàn, thuận lợi nhất cho người dân", ông Phương nói và cho rằng trong bối cảnh nhu cầu đi lại của người dân rất lớn hiện nay, việc để hành khách đến sân bay và di chuyển tiếp là một phương án khả thi, an toàn hơn so với đi bằng đường bộ và các phương tiện cá nhân.
Phân tích nguyên nhân khiến các địa phương e ngại việc mở lại đường bay nội địa, TS Trần Đắc Phu, Cố vấn cao cấp Trung tâm Đáp ứng khẩn cấp sự kiện y tế công cộng, nói do tỷ lệ tiêm vaccine cả nước còn thấp và không đồng đều, trong khi người đã tiêm đủ hai liều vẫn còn khả năng nhiễm bệnh.
Theo ông, để mở lại hàng không, trước mắt cần quy định riêng cho hành khách từ vùng nguy cơ cao. Khi Việt Nam tiêm chủng toàn quốc với tỷ lệ cao sẽ bỏ các hàng rào xét nghiệm, cách ly. "Tiêu chí kiếm soát dịch cần thống nhất giữa các địa phương, không để mỗi nơi làm một kiểu dẫn đến mở chỗ này lại nghẽn chỗ kia" ông Phu nói.
Phó tổng giám đốc Bamboo Airways Nguyễn Mạnh Quân dẫn kinh nghiệm nhiều nước trên thế giới, cho thấy quy định mức cao nhất cho phép hành khách đi lại là tiêm đủ vaccine hai mũi, có xét nghiệm trong 72 tiếng trước khi xuất phát và cách ly khi tới nơi.
Hiện Việt Nam đã có hơn 13 triệu người tiêm đủ 2 mũi. Do vậy ở giai đoạn một, với các đường bay nội địa có thể cho phép người đã tiêm đủ liều, xét nghiệm âm tính được tham gia dịch vụ vận tải hàng không.
Ông Quân cũng cho rằng, đường bay nội địa dài nhất chỉ mất 2,5 giờ hành trình nên sẽ không dẫn tới nguy cơ như trên đường bộ kéo dài nhiều ngày. Sau khi có bộ tiêu chí thống nhất dành cho hành khách, tùy theo mức độ chống dịch ở từng đia phương để điều chỉnh quy định cho các sân bay. 
Từ 10/10, Cục Hàng không Việt Nam dự kiến mở lại 10 đường bay nội địa giai đoạn một. Đó là các đường bay giữa TP HCM với Thanh Hóa/Khánh Hòa/Phú Yên/Bình Định/Phú Quốc với tần suất 4 chuyến khứ hồi mỗi ngày. Đường bay TP HCM với Thừa Thiên Huế có một chuyến khứ hồi mỗi tuần; TP HCM với Nghệ An có 2 chuyến mỗi tuần. Đường bay Thanh Hóa - Khánh Hòa/Buôn Ma Thuột/Phú Quốc có một chuyến mỗi ngày.
Hành khách khi bay phải tuân thủ 5K, khai báo y tế tại điểm xuất phát và điểm đến, có chứng nhận tiêm đủ liều vaccine trong ít nhất 14 ngày, hoặc có giấy chứng nhận khỏi bệnh Covid-19 không quá 6 tháng tính đến thời điểm xuất phát. Với hành khách từ vùng được đánh giá là "nguy cơ rất cao tương ứng với màu đỏ (cấp 4)" phải có xét nghiệm âm tính trong 72 tiếng trước giờ khởi hành.
Trong số các tỉnh, thành đồng ý mở lại đường bay nội địa hiện chưa có Hà Nội. Lãnh đạo thành phố nêu lo ngại khi mở lại ngay vận tải hành khách hàng không, sẽ có 385 chuyến bay mỗi ngày trên toàn quốc (thực tế dự kiến đến sân bay Nội Bài là 90 chuyến). Tương ứng với số chuyến bay là hàng chục nghìn hành khách từ khắp cả nước, dẫn đến nguy cơ rất lớn về dịch bệnh.
Trong khi đó, công suất các khu cách ly tập trung trên địa bàn Hà Nội hiện là 110.000 người, nên có thể chỉ trong thời gian ngắn mở cửa trở lại hàng không và đường sắt, số người cần cách ly tập trung sẽ vượt năng lực của thành phố."""

text3 = """Ở Việt Nam, công nghiệp hóa, hiện đại hóa là một trong những giải pháp quyết định đưa nước ta sớm thoát khỏi tình trạng nghèo nàn lạc hậu, khắc phục nguy cơ tụt hậu về kinh tế, tiến kịp với các nước trong khu vực và trên thế giới, đồng thời cải thiện đời sống của nhân dân, tăng cường tiềm lực quốc phòng - an ninh, củng cố vững chắc độc lập chủ quyền của Tổ quốc. Qua đó giải phóng những tiềm lực to lớn của đất nước và mở ra trước mắt chúng ta những sự thay đổi thần kỳ trong phát triển kinh tế giúp nước ta vươn lên trở thành nước có mức thu nhập trung bình thấp, vượt qua thời kỳ khủng hoảng kinh tế -xã hội, đói nghèo và kém phát triển. Đặc biệt, cuộc cách mạng công nghiệp lần thứ tư được cho là đã bắt đầu từ vài năm gần đây là cuộc cách mạng về sản xuất thông minh dựa trên các thành tựu đột phá trong các lĩnh vực công nghệ thông tin, công nghệ sinh học, công nghệ nano…với nền tảng là các đột phá của công nghệ số. Cách mạng công nghiệp lần thứ tư sẽ mở ra cơ hội phát triển cho công nghiệp hóa, hiện đại hóa ở Việt Nam. 
Chính vì thế, việc nhìn nhận và đánh giá thực trạng của công nghiệp hóa, hiện đại hóa, hội nhập quốc tế hiện nay là một vấn đề hết sức quan trọng đối với Đảng ta. Đảng phải biết cách vận dụng sáng tạo trong trong việc đẩy mạnh công nghiệp hóa, hiện đại hóa để phát triển kinh tế - xã hội  tạo điều kiện tăng cường củng cố an ninh - quốc phòng và là tiền đề cho việc xây dựng một nền kinh tế độc lập, tự chủ, đủ sức tham gia một cách có hiệu quả vào sự phân công và hợp tác quốc tế.
Vì lẽ đó, trong báo cáo lần này nhóm trình bày về những quan điểm và giải pháp của Đảng trong việc góp phần đẩy mạnh công nghiệp hóa, hiện đại hóa, hội nhập quốc tế ở Việt Nam hiện nay, cũng như áp dụng những quan điểm, chủ trương của Đảng và nhà nước để nhìn nhận các vấn đề, thực trạng trong xã hội Việt Nam hiện nay.
Trong đề tài này nhóm sẽ tập trung rõ các vấn đề cốt lõi trong quá trình đẩy mạnh công nghiệp hóa, hiện đại hóa, hội nhập quốc tế ở Việt Nam hiện nay.
Hiểu rõ và vận dụng được quan điểm chỉ đạo của Đảng về công nghiệp hóa, hiện đại hóa và hội nhập quốc tế đất nước.
Đánh giá, phân tích thực trạng và đề xuất các giải pháp cần thiết cho trong quá trình công nghiệp hóa, hiện đại hóa, hội nhập quốc tế ở Việt Nam hiện nay.
Nhằm tập trung vào vấn đề trung tâm của đề tài nên nhóm đã xác định ra phạm vi nghiên cứu là xác định tìm giải pháp cho giải quyết vấn đề hiện tại để tiến hành hành công nghiệp hóa, hiện đại hóa, hội nhập quốc tế phạm vi địa bàn TP.Hồ Chí Minh, trong vòng 10 năm trở lại đây để đề xuất giải pháp thực hiện từ 2021 trở về sau.
Bước vào thời kỳ đổi mới trên cơ sở phân tích khoa học các điều kiện trong nước và quốc tế, Đảng đã nêu ra những quan điễm chĩ đạo quá trình thực hiện công nghiệp hóa, hiện đại hóa đất nước trong điều kiện mới. Những quan điểm này được hội nghị lần thứ bảy Ban Chấp hành Trung ương khóa VII nêu ra và được phát triên bổ sung qua các Đại hội VIII, IX, X, XI, XII của Đảng. Những quan điễm đó được làm rõ như sau.
Lịch sử công nghiệp hóa trên thế giới đã trải qua hàng trăm năm. Vào giữa thế kỷ XVIII, một số nước phương Tây, mở đầu là nước Anh đã tiến hành cuộc cách mạng công nghiệp, với nội dung chủ yếu là chuyển từ lao động thủ công sang lao động cơ khí. Đây là mốc đánh dấu sự khởi đầu cho tiến trình công nghiệp hóa của thế giới. Tuy vậy, phải đến thế kỷ XIX, khái niệm công nghiệp hóa mới được dùng để thay thế cho khái niệm cách mạng công nghiệp, mặc dù sau cách mạng công nghiệp ở Anh, một thế hệ công nghiệp hóa đã diễn ra ở các nước Tây Âu, Bắc Mỹ và Nhật Bản. Có thể khái quát, công nghiệp hóa là quá trình tạo sự chuyển biến từ nền kinh tế nông nghiệp với nền kinh tế lạc hậu, dựa trên lao động thủ công, năng suất thấp sang nền kinh tế công nghiệp với cơ cấu kinh tế hiện đại, dựa trên lao động sử dụng bằng máy móc, tạo ra năng suất lao động cao. Như vậy, công nghiệp hóa là quá trình biến một nước có nền kinh tế lạc hậu thành nước công nghiệp hiện đại với trình độ công nghệ, kỹ thuật tiên tiến, có năng suất lao động cao trong các ngành kinh tế quốc dân. Hiện đại hóa là quá trình tận dụng mọi khả năng để đạt trình độ công nghệ ngày càng tiên tiến, hiện đại.
Trong điều kiện của Việt Nam, Đảng ta xác định: “Công nghiệp hóa, hiện đại hóa là quá trình chuyển đổi căn bản, toàn diện các hoạt động sản xuất, kinh doanh, dịch vụ và quản lý kinh tế - xã hội từ sử dụng lao động thủ công là chính sang sử dụng một cách phổ biến sức lao động cùng với công nghệ, phương tiện, phương pháp tiên tiến, hiện đại dựa trên sự phát triển của công nghiệp và tiến bộ khoa học - công nghệ, tạo ra năng suất lao động xã hội cao”.
Quan điểm đẩy mạnh công nghiệp hoá, hiện đại hoá (CNH, HĐH) gắn với phát triển kinh tế tri thức (KTTT) đã được Đảng ta chính thức đề cập tại Đại hội X: “Tranh thủ cơ hội thuận lợi do bối cảnh quốc tế tạo ra và tiềm năng, lợi thế của nước ta để rút ngắn quá trình CNH, HĐH đất nước theo định hướng xã hội chủ nghĩa gắn với phát triển KTTT, coi KTTT là yếu tố quan trọng của nền kinh tế và CNH, HĐH. Phát triển mạnh các ngành và sản phẩm kinh tế có giá trị gia tăng cao dựa nhiều vào tri thức; kết hợp việc sử dụng nguồn vốn tri thức của con người Việt Nam với tri thức mới nhất của nhân loại” . Trong Cương lĩnh xây dựng đất nước trong thời kỳ quá độ lên chủ nghĩa xã hội (bổ sung, phát triển năm 2011) đã phát triển và cụ thể hoá thêm một bước quan điểm đẩy mạnh CNH, HĐH gắn với phát triển KTTT: “Phát triển kinh tế là nhiệm vụ trung tâm; thực hiện CNH, HĐH đất nước gắn với phát triển KTTT; …Phát triển khoa học và công nghệ nhằm mục tiêu đẩy mạnh CNH, HĐH đất nước, phát triển KTTT, vươn lên trình độ tiên tiến của thế giới”.
Thực tế lịch sử đã chứng minh, trong thời đại toàn cầu hoá và hội nhập quốc tế, tất cả các nước có xuất phát điểm thấp, muốn vươn lên trình độ tiên tiến của thế giới không có con đường nào khác ngoài việc thực hiện CNH, HĐH gắn với phát triển KTTT. Đây chính là xu thế khách quan của thời đại toàn cầu hoá; đồng thời cũng là con đường “rút ngắn” của quá trình công nghiệp hoá theo hướng hiện đại nhằm chuyển nền kinh tế nông nghiệp thành nền kinh tế công nghiệp - tri thức - và nền KTTT.
Thực ra, từ những thập kỷ cuối thế kỷ XX cho tới nay, khoa học và công nghệ đã có những bước phát triển kỳ diệu, đặc biệt là sự bùng nổ và sự hội tụ của công nghệ thông tin, công nghệ sinh học, công nghệ nanô, đúng như những tiên đoán của C.Mác và Ph.Ăngghen từ giữa thế kỷ XIX, “tri thức sẽ trở thành lực lượng sản xuất trực tiếp”; “giá trị của lao động cơ bắp trong sản phẩm làm ra sẽ giảm còn cực nhỏ”; “lực lượng sản xuất tinh thần”; “sự xuất hiện công nhân khoa học”... thúc đẩy mạnh mẽ quá trình chuyển đổi từ nền kinh tế công nghiệp sang  nền KTTT.
Thuật ngữ “kinh tế tri thức” (knowledge economy), mặc dù xuất hiện từ khá sớm, nhưng  mãi tới năm 1990 mới chính thức được ghi nhận trong báo cáo của Liên hiệp quốc và từ đó trên sách báo đã xuất hiện những định nghĩa khác nhau về KTTT. Năm 1996, OECD đưa ra định nghĩa: “Nền KTTT là nền kinh tế trong đó sự sản sinh ra, phổ cập và sử dụng tri thức giữ vai trò quyết định nhất đối với sự phát triển kinh tế, tạo ra của cải, nâng cao chất lượng cuộc sống” .
Năm 2000, APEC cũng quan niệm: Nền KTTT là nền kinh tế trong đó sự sản sinh ra, truyền bá và sử dụng tri thức là động lực chủ yếu nhất của sự tăng trưởng, tạo ra của cải, tạo ra việc làm trong tất cả các ngành kinh tế.
UNDP và Ngân hàng thế giới đưa ra định nghĩa: Nền KTTT là nền kinh tế sử dụng có hiệu quả tri thức cho phát triển kinh tế và xã hội, bao gồm cả việc khai thác kho tri thức toàn cầu, cũng như làm chủ và sáng tạo tri thức cho những nhu cầu của riêng mình.
Dù còn có những định nghĩa khác nhau, song có thể nhận diện nền KTTT là nền kinh tế sử dụng có hiệu quả tri thức cho phát triển kinh tế và xã hội; đó là nền kinh tế với những đặc trưng chủ yếu sau.
Khác với nền kinh tế nông nghiệp chủ yếu phụ thuộc vào tự nhiên, chủ thể chính là nông dân cần cù trong sử dụng “cày chìa vôi” để sản xuất; nền kinh tế công nghiệp, chủ thể là công nhân thành thạo trong thao tác các công cụ cơ khí, nền KTTT là nền kinh tế mà chủ thể của nó là công nhân - trí thức sáng tạo trong điều khiển các công cụ tự động hoá, các công cụ truyền thông đa phương tiện, sử dụng có hiệu quả tri thức và công nghệ để phát triển kinh tế, xã hội và bảo vệ môi trường sinh thái.
Hoạt động chủ yếu nhất trong nền KTTT là hoạt động tạo ra, quảng bá và sử dụng hiệu quả tri thức, biến tri thức thành giá trị. Tri thức đã trở thành nguồn lực hàng đầu, mà giá trị của nó chiếm tỷ trọng cao, thậm chí là tuyệt đối trong sản phẩm. Với nền KTTT, nhân tài trí thức được coi trọng, quyền  sở hữu trí tuệ trở thành yếu tố quan trọng; tri thức trở thành hình thức cơ bản nhất của vốn, quan trọng hơn cả tài nguyên, sức lao động; lực lượng sản xuất từ dựa vào tài nguyên thiên nhiên chuyển dần sang dựa chủ yếu vào năng lực trí tuệ của con người.
Nền KTTT phát huy tối đa  năng lực sáng tạo của con người trong vận dụng và sáng tạo ra tri thức mới, thúc đẩy tăng trưởng kinh tế, phát triển con người và xã hội một cách nhanh chóng. Bởi vậy, vòng đời của một sản phẩm, một công nghệ từ lúc nảy sinh, phát triển, chín muồi đến tiêu vong ngày càng rút ngắn; trước đây vòng đời công nghệ tính bằng nhiều thập niên, ngày nay tính bằng năm, thậm chí tính bằng tháng. Tốc độ đổi mới công nghệ rất nhanh chóng.
Sau 25 năm vượt qua các thử thách và khó khăn to lớn, tiến hành công cuộc đổi mới trên mọi mặt, đến nay Việt Nam có đủ cơ sở để khẳng định rằng, đổi mới là sự lựa chọn đúng đắn, đã đặt Việt Nam vào quỹ đạo phát triển mới về chất, phù hợp với xu thế của thời đại và ý nguyện của nhân dân.
Ngày nay, công cuộc đổi mới đã chuyển sang giai đoạn phát triển mới: đổi mới và hội nhập; đổi mới để giữ vững ổn định chính trị một cách tích cực cho phát triển bền vững; đổi mới hệ thống chính trị phải tương thích với đổi mới kinh tế nhằm xây dựng và hoàn thiện nền dân chủ xã hội chủ nghĩa bảo đảm toàn bộ quyền lực thuộc về nhân dân và đổi mới tạo nền tảng để đến năm 2020 nước ta cơ bản trở thành nước công nghiệp theo hướng hiện đại.
Đổi mới cũng là quá trình vừa làm vừa tìm tòi, rút kinh nghiệm; kinh nghiệm của chính bản thân chúng ta và kinh nghiệm của thế giới. Đó là quá trình tiếp thu có chọn lọc hết sức nghiêm túc và sáng tạo. Một phần tư thế kỷ qua, đổi mới không chỉ là một ý chí, mong muốn, một khẩu hiệu tuyên truyền mà là quyết tâm chính trị - hành động lan tỏa, thấm sâu trong mọi tế bào của đời sống xã hội, làm thay đổi nếp nghĩ, cách làm và gây dựng cơ nghiệp. Đổi mới đã làm thay đổi căn bản và toàn diện trên tất cả các lĩnh vực từ kinh tế, chính trị, văn hóa, xã hội đến an ninh, quốc phòng và đối ngoại: Kinh tế tăng trưởng nhanh, đạt tốc độ bình quân 7,2%/năm. Năm 2010, tổng sản phẩm trong nước bình quân đầu người đạt 1.200 USD. Cơ cấu kinh tế chuyển dịch theo hướng tích cực. Thể chế kinh tế thị trường định hướng xã hội chủ nghĩa tiếp tục được hoàn thiện. Các lĩnh vực văn hoá, xã hội đạt thành tựu quan trọng trên nhiều mặt. Đời sống vật chất và tinh thần của nhân dân được cải thiện rõ rệt; dân chủ trong xã hội tiếp tục được mở rộng. Chính trị - xã hội ổn định; quốc phòng, an ninh được giữ vững. Công tác đối ngoại, hội nhập quốc tế được triển khai sâu rộng và hiệu quả góp phần tạo môi trường hòa bình, ổn định và tăng thêm nguồn lực cho phát triển đất nước.
Diện mạo của đất nước có nhiều thay đổi. Thế và lực của đất nước vững mạnh thêm nhiều; vị thế của Việt Nam trên trường quốc tế được nâng lên, tạo ra những tiền đề quan trọng để đẩy nhanh công nghiệp hoá, hiện đại hóa đất nước và nâng cao chất lượng cuộc sống của nhân dân.
Những thành tựu to lớn và có ý nghĩa lịch sử ấy đã nâng cao tầm vóc và vị thế của đất nước ta, ngày càng được bạn bè quốc tế đánh giá cao và nhiệt tình ủng hộ. Tuy vậy, kinh tế phát triển chưa bền vững; chất lượng, hiệu quả, sức cạnh tranh thấp, chưa tương xứng với tiềm năng, cơ hội và yêu cầu phát triển của đất nước. Nền kinh tế chủ yếu vẫn dựa vào tài nguyên và lao động, giá trị do tri thức tạo ra không đáng kể; chưa khơi dậy và phát huy tối đa năng lực sáng tạo của con người; nền kinh tế còn chứa đựng nhiều yếu tố phát triển không bền vững, có nguy cơ tụt hậu xa hơn so với các nước khác. Về cơ bản nước ta vẫn là nước chưa phát triển.
Là một nước nông nghiệp đi lên, trong bối cảnh toàn cầu hoá và hội nhập quốc tế để phát triển, nền KTTT cần tiến hành đồng thời và lồng ghép của hai quá trình: quá trình chuyển từ kinh tế nông nghiệp lên kinh tế công nghiệp và quá trình chuyển từ kinh tế công nghiệp lên KTTT, trong đó nhất thiết phải bắc được những “nhịp cầu trung gian” để thực hiện những bước chuyển quá độ. Đây là cả một sự nghiệp vô cùng khó khăn, phức tạp. Trong khi đối với các nước đi trước, để thực hiện bước chuyển này chỉ là hai quá trình kế tiếp nhau, còn đối với nước ta để thực hiện bước phát triển “rút ngắn” đòi hỏi phải kết hợp các bước đi tuần tự với các bước phát triển nhảy vọt. Một mặt, tận dụng lao động, cơ sở vật chất hiện có, sử dụng tri thức mới, công nghệ mới để phát triển sản xuất, tạo việc làm, tăng thu nhập. Mặt khác, đi thẳng vào hiện đại ở những khâu, những lĩnh vực, những ngành có lợi thế, phát triển các ngành, các vùng kinh tế mũi nhọn tạo thành đầu tầu có sức kéo lôi mạnh toàn bộ nền kinh tế đi lên.
Bởi vậy, việc chuyển nền kinh tế theo hướng phát triển chủ yếu dựa vào tri thức trở thành yêu cầu cấp thiết, không thể trì hoãn. Bỏ lỡ thời cơ lớn này, Việt Nam sẽ tụt hậu xa hơn và đó là hiểm họa của dân tộc. Báo cáo chính trị tại Đại Hội X của Đảng đã chỉ ra: “Tranh thủ cơ hội thuận lợi do bối cảnh quốc tế tạo ra và tiềm năng, lợi thế của nước ta để rút ngắn quá trình CNH, HĐH đất nước theo định hướng XHCN gắn với phát triển KTTT, coi KTTT là yếu tố quan trọng của nền kinh tế và CNH, HĐH. Phát triển mạnh các ngành và sản phẩm kinh tế có giá trị gia tăng cao dựa nhiều vào tri thức; kết hợp việc sử dụng nguồn vốn tri thức của con người Việt Nam với tri thức mới nhất của nhân loại” .  Đại hội XI, trên tinh thần tiếp tục nâng cao năng lực lãnh đạo và sức chiến đấu của Đảng, phát huy sức mạnh toàn dân tộc, đẩy mạnh toàn diện công cuộc đổi mới, tạo nền tảng để đến năm 2020 nước ta cơ bản trở thành nước công nghiệp theo hướng hiện đại, đã khẳng định: Để xây dựng thành công chủ nghĩa xã hội và bảo vệ vững chắc Tổ quốc xã hội chủ nghĩa, phải tiếp tục đổi mới toàn diện, mạnh mẽ với chất lượng và hiệu quả cao hơn; đẩy mạnh CNH, HĐH gắn với phát triển KTTT; phát triển kinh tế thị trường gắn với giải quyết hài hoà các vấn đề xã hội, môi trường...
Trong vài ba thập kỷ đầu của thế kỷ XXI, để thực hiện có kết quả bước phát triển “rút ngắn” sớm xác lập nền KTTT Việt Nam, cần thiết phải tiến hành đồng bộ nhiều giải pháp, trong đó cần tập trung toàn lực vào việc thực hiện những giải pháp mang tính đột phá sau.
Một là, tiếp tục cải cách mạnh mẽ, có hiệu quả nền giáo dục và đào tạo theo hướng nâng cao dân trí, đào tạo nhân lực, bồi dưỡng và phát huy nhân tài.
Cải cách triệt để nền giáo dục được coi là “đột phá khẩu”, bởi nó hướng tới việc đào tạo, bồi dưỡng và phát huy nguồn nhân lực có chất lượng cao cho quá trình tái cơ cấu lại nền kinh tế, tăng lợi thế cạnh tranh, bảo đảm phát triển nhanh, hiệu quả, bền vững kinh tế - xã hội, đẩy mạnh CNH, HĐH gắn với phát triển KTTT. Dự thảo Chiến lược phát triển kinh tế - xã hội 2011 - 2020 khẳng định: “Phát triển và nâng cao chất lượng nguồn nhân lực, nhất là nguồn nhân lực chất lượng cao là một đột phá chiến lược, là yếu tố quyết định quá trình cơ cấu lại nền kinh tế, chuyển đổi mô hình tăng trưởng và là lợi thế cạnh tranh dài hạn, bảo đảm kinh tế - xã hội phát triển nhanh, hiệu quả, bền vững... Thực hiện các chương trình, đề án đào tạo nhân lực chất lượng cao đối với các ngành, lĩnh vực chủ yếu, mũi nhọn. Chú trọng phát hiện, bồi dưỡng, phát huy nhân tài; đào tạo nhân lực cho phát triển KTTT”.
Hai là, phát triển mạnh khoa học, công nghệ, chú trọng vào việc việc ứng dụng, sáng tạo công nghệ cao làm động lực đẩy nhanh quá trình CNH, HĐH, phát triển KTTT.
Đã từ lâu, chúng ta quan tâm tới việc phát triển khoa học và công nghệ, song trong lĩnh vực công nghệ chúng ta chưa tập trung vào việc vận dụng và sáng tạo, phát triển công nghệ cao. Thực tế cho thấy, việc thiếu cơ chế chính sách trong việc tập trung cho công nghệ cao không tạo động lực cho phát triển; còn công nghệ lạc hậu sẽ cản trở, kìm hãm sự phát triển của CNH, HĐH. Đã đến lúc cần phải “tuyên chiến” với công nghệ lạc hậu, chỉ nhập khẩu, ứng dụng và đầu tư sáng tạo công nghệ cao trên cơ sở sử dụng có hiệu quả những công nghệ hiện có. Là nước đi sau, Việt Nam có thể bỏ qua các thế hệ công nghệ trung gian để đi ngay vào các công nghệ tiên tiến, công nghệ cao, không lệ thuộc vào cơ sở hạ tầng đã có. Trong các lĩnh vực công nghiệp dựa vào tri thức, nhất là công nghệ thông tin, Việt Nam có thể chọn một số lĩnh vực để bứt phá lên trước.
Nhiệm vụ trung tâm là sử dụng tri thức mới, công nghệ mới của thời đại kết hợp với sáng tạo tri thức mới, công nghệ mới để nhanh chóng chuyển dịch cơ cấu kinh tế theo hướng tăng nhanh các ngành công nghiệp và dịch vụ hàm lượng tri thức cao, giá trị gia tăng cao. Đây chính là yêu cầu và nội dung của công cuộc CNH, HĐH gắn với phát triển KTTT trong thời đại toàn cầu hoá với một trật tự “thế giới phẳng”.
Báo cáo chính trị của Ban Chấp hành Trung ương Đảng khóa X tại Đại hội đại biểu toàn quốc lần thứ XI, khẳng định: “Phát triển mạnh khoa học, công nghệ làm động lực đẩy nhanh quá trình CNH, HĐH, phát triển KTTT; góp phần tăng nhanh năng suất, chất lượng, hiệu quả, sức cạnh tranh của nền kinh tế, sự phát triển nhanh, bền vững của đất nước; nâng tỉ lệ đóng góp của yếu tố năng suất tổng hợp vào tăng trưởng. Thực hiện đồng bộ ba nhiệm vụ chủ yếu: nâng cao năng lực khoa học, công nghệ; đổi mới cơ chế quản lý; đẩy mạnh nghiên cứu ứng dụng” .
Ba là, tiếp tục đổi mới, phát huy vai trò quản lý, điều hành ở tầm vĩ mô của Nhà nước trong cải cách giáo dục - đào tạo, phát triển khoa học, công nghệ và trong quản lý và phát triển kinh tế thị trường - xã hội.
Vai trò quản lý của Nhà nước trong điều kiện mới phải được xem như là một quá trình tạo điều kiện cho việc thúc đẩy phát triển chứ không đơn thuần là sự thực thi những quyết sách không sát với thực tiễn.
Từ Hội Nghị Trung ương 10/ khoá IX, với chủ trương gắn phát triển kinh tế là nhiệm vụ trung tâm với xây dựng Đảng là khâu then chốt và xây dựng nền văn hoá tiên tiến đậm đà bản sắc dân tộc làm nền tảng tinh thần cho phát triển nhanh, hiệu quả và bền vững ở nước ta, Đảng ta cũng đồng thời chỉ ra và khẳng định ba loại động lực chính cho phát triển: phát triển kinh tế thị trường định hướng XHCN; nâng cao năng lực và sức chiến đấu của Đảng và xây dựng; phát triển nền văn hoá tiên tiến đậm đà bản sắc dân tộc. Dù trên lĩnh vực nào thì yêu cầu sáng tạo và đổi mới cũng luôn là năng lực nội sinh của quá trình phát triển kinh tế - xã hội. Bởi vậy, tiếp tục đổi mới cũng đồng nghĩa với tiếp tục đẩy mạnh CNH, HĐH gắn với phát triển KTTT tạo nên những điều kiện và động lực to lớn cho phát triển nhanh, hiệu quả và bền vững.
Trong tổng thể những nội dung đổi mới toàn diện, cần tập trung trọng tâm vào việc nhận thức lại và thực hiện đúng chức năng, vai trò của nhà nước đối với phát triển kinh tế, từ điều khiển, chỉ huy sang là “kiến trúc sư” trong cải cách giáo dục - đào tạo, trong phát triển khoa học, công nghệ, trong việc tạo môi trường thuận lợi và động viên mọi nguồn lực để thực hiện bước chuyển đổi từ nền kinh tế công nghiệp sang nền KTTT. Hơn nữa, cần tăng cường tính minh bạch của chính phủ; phòng chống tham nhũng có hiệu quả, tạo điều kiện cho nhân dân tham gia tích cực, sáng tạo trong việc hoạch định và thực thi chính sách. Trong điều kiện hiện nay, với sự biến đổi nhanh chóng, khó lường của tình hình thế giới; sự phát triển nhanh chóng của khoa học và công nghệ, để phát huy vai trò tạo điều kiện của nhà nước trong đổi mới và phát triển đòi hỏi công tác chỉ đạo, quản lý, điều hành của Nhà nước phải hết sức nhạy bén, kiên quyết, sáng tạo, bám sát thực tiễn đất nước; nâng cao chất lượng hoạt động dự báo, kịp thời đề ra các giải pháp phù hợp với tình hình mới.
Quá trình công nghiệp hóa, hiện đại hóa ở nước ta đang trong quá trình phát triển mạnh mẽ. Không những ở thành thị mà cả ở nông thôn. Quá trình này đã đem lại những thành tựu to lớn cho đất nước. Song chúng ta cũng đã thấy rõ vấn đề môi trường tự nhiên đang bị ô nhiễm nặng, gây tác động xấu đến sản xuất và đời sống. Những khu công nghiệp, cụm điểm công nghiệp ngày càng nhiều, những khu đô thị mới hiện đại mọc lên, và sản xuất hàng hóa, chế biến nông sản, thực phẩm ở nông thôn ngày càng trở nên sôi động. Do đó, việc giải quyết vấn đề môi trường thông qua các chính sách phát triển kinh tế đi đôi với bảo vệ môi trường đang là vấn đề cấp thiết được Nhà nước đặt ra hiện nay.
Ngày 9/6/2014, Thủ tướng Chính phủ đã ký quyết định số 879/QĐ-TTg phê duyệt chiến lược phát triển công nghiệp Việt Nam đến năm 2025, tầm nhìn đến 2035. Trong đó liên quan đến môi trường thì quan điểm phát triển đã nêu rõ: phát triển công nghiệp trên cơ sở tăng trưởng xanh, phát triển bền vững bảo vệ môi trường.
Nước nào cũng vậy, phát triển công nghiệp thường kèm theo nguy cơ gây ra ô nhiễm môi trường. Nếu không quan tâm phòng, chống ngay từ đầu thì hiểm họa sẽ khôn lường. Trên thực tế đã có những quốc gia phát triển công nghiệp quá nhanh đã phải trả giá đắt cho vấn đề này. Quá trình công nghiệp hóa thường kèm theo tốc độ đô thị hóa nhanh, mức độ phát thải lớn, tốc độ khai thác tài nguyên thiên nhiên cũng tăng đến mức tối đa và kết cục là môi trường bị hủy hoại do ô nhiễm mạnh, tài nguyên thì cạn kiệt ảnh hưởng đến sức khỏe cộng đồng và đe dọa toàn bộ hệ sinh thái. Sau gần 30 năm đổi mới, công nghiệp Việt Nam có tốc độ tăng trưởng trung bình trên 10%/năm, đóng góp tích cực cho sự phát triển kinh tế - xã hội của đất nước. Song, công nghiệp cũng là một những ngành tạo ra nhiều chất thải nhất, ảnh hưởng tiêu cực đến môi trường sinh thái. Những ngành công nghiệp có mức độ gây ô nhiễm cao như  khai thác than và khoáng sản; công nghiệp sản xuất điện; công nghiệp hóa chất; công nghiệp luyện kim; công nghiệp thực phẩm, dệt may, da giầy… khiến cho Chính phủ phải đặt ra những bài toán kèm theo lời giải mang tính hiệu quả cao. Chính sách cho bảo vệ môi trường trong quá trình công nghiệp hóa hiện nay thực sự là một vấn đề hết sức cấp thiết và đòi hỏi những chính sách đó phải vừa đảm bảo cho mục tiêu công nghiệp hóa, hiện đại hóa đạt được thành công vừa đảm bảo cho môi trường sinh thái đạt ngưỡng an toàn.
Cụm từ “Phát triển bền vững” đã và đang trở thành tuyên ngôn và chiến lược hành động của nhiều quốc gia trên thế giới. Kể từ Hội nghị Stockhom năm 1987 đến Hội nghị Rio de Janeiro năm 1992 và Johanesbour năm 2002 thì trên 200 quốc gia tham gia đã xác định vấn đề môi trường là yếu tố quyết định cho phát triển bền vững. Ngày nay, bên cạnh những thành tựu kinh tế thì thế giới đang phải đối mặt với nhiều thách thức về đói nghèo, thiên tai, thiếu năng lượng và nguyên liệu, bệnh tật và suy thoái môi trường. Các nước ngày càng nhận thức rõ hơn các nguy cơ tiềm ẩn đe dọa đến sự phát triển bền vững, hệ quả của quá trình công nghiệp hóa thiếu cân nhắc về môi trường. Các chính phát triển bền vững sẽ giúp cho các ngành kinh tế có sự cân nhắc sử dụng các nguồn lực sao cho tiết kiệm và hiệu quả nhất. Các chính sách phát triển bền vững là công cụ điều chỉnh hướng tới những tư duy cân bằng hơn trong cách thức giải quyết và đáp ứng các mục tiêu kinh tế - xã hội và môi trường, nhằm tạo ra những tiền đề phát triển trong tương lai.
Việt Nam là một trong những quốc gia sớm ký tuyên bố chung về thực hiện phát triển bền vững từ năm 1992. Cuối năm 1993, luật Bảo vệ môi trường được ban hành cùng với sự tích cực chuẩn bị Chương trình nghị sự 21 của Việt Nam đã thể hiện quyết tâm đối với các cam kết quốc tế. Những nỗ lực đó của Việt nam được cộng đồng quốc tế đánh giá cao. Tuy nhiên, đó chỉ là những bước khởi đầu của phát triển bền vững, vì Việt nam vẫn đang là nước đang phát triển có thu nhập trung bình thấp, nên nỗi lo toàn về kinh tế đang bao trùm, chi phối các lĩnh vực đời sinh – xã hội và đặc biệt là sản xuất công nghiệp.
Bước vào thời kỳ công nghiệp hóa với sự tăng trưởng nhanh của công nghiệp, những vấn đề môi trường cũng đang tích tụ và tăng nhanh làm cộng đồng lo ngại. Quá trình hội nhập kinh tế cũng có những tác động làm thay đổi nhận thức của doanh nghiệp về hệ quả môi trường và cạnh tảnh, và nhiều nguy cơ tiềm ẩn khác như khủng hoảng năng lượng, thiếu hút nguyên liệu, mất cân bằng sinh thái và biến đổi khí hậu toàn cầu. Trước đòi hỏi bức xúc về môi trường trong quá trình công nghiệp hóa, hiện đại hóa đất nước, từ năm 2004, Bộ Chính trị đã kịp thời ra chỉ thị về công tác bảo vệ môi trường trong thời kỳ đẩy mạnh công nghiệp hóa, hiện đại hóa đất nước. Trong đó xác định rõ: bảo vệ môi trường là sự nghiệp của toàn Đảng, toàn dân và toàn quân; bảo vệ  môi trường là một nội dung cơ bản không thể tách rời trong đường lối, chủ trương và kế hoạch phát triển kinh tế - xã hội của tất cả các cấp, các ngành, là cơ sở quan trọng đảm bảo phát triển bền vững, thực hiện thắng lợi sự nghiệp công nghiệp hóa, hiện đại hóa đất nước; coi phòng ngừa và ngăn chặn ô nhiễm là nguyên tắc chủ đạo kết hợp với xử lý ô nhiễm, cải thiện môi trường và bảo tồn thiên nhiên. Cũng đầu năm 2004, chiến lược quốc gia về bảo vệ môi trường và tiếp theo là Định hướng chiến lược phát triển bền vững của Việt Nam đã được Chính phủ thông qua.
Hiện nay, biến đổi khí hậu toàn cầu không còn là dự báo mà đã trở thành mối đe dọa gây ra nhiều thảm họa và tai biến thiên nhiên trên toàn thế giới. Việt Nam là một trong những quốc gia được dự báo sẽ bị tác động nghiêm trọng nhất do biến đổi khí hậu và mực nước biển dâng. Việt nam không thuộc nhóm nước phải cam kết cắt giảm khí nhà kính. Nhưng xét về lợi ích toàn cầu và của quốc gia, Việt Nam cần chủ động phát triển công nghiệp xanh phù hợp với mục tiêu chung về phát triển kinh tế - xã hội, nhằm đạt 3 mục tiêu: xóa đói, giảm nghèo, giảm phát khí thải nhà kính và chủ động ứng phó với biến đổi khí hậu.
Công nghiệp xanh đang được ứng dụng và phát triển mạnh mẽ ở Mỹ, Châu Âu, Nhật Bản, Hàn Quốc… Công nghiệp Xanh là một chiến lược hoạt động mà các ngành công nghiệp tại các quốc gia ở tất cả các giai đoạn phát triển có thể sử dụng để đạt được phát triển bền vững bằng cách tách tăng trưởng kinh tế ra khỏi việc sử dụng quá mức tài nguyên và gây ô nhiễm môi trường. Các ngành công nghiệp xanh sẽ giảm thiểu lượng chất thải phát thải, sử dụng năng lượng một cách hiệu quả và triển khai sử dụng các nguồn tài nguyên có thể tái tạo làm nguyên liệu cũng như năng lượng đầu vào. Ngoài ra, các ngành công nghiệp xanh còn cung cấp hàng hóa và dịch vụ môi trường, như quản lý chất thải, năng lượng tái tạo,… theo cách có thể dự đoán được để đảm bảo rằng môi trường làm việc, cộng đồng địa phương và thiên nhiên nói chung được an toàn trước những rủi ro nguy hiểm về môi trường. Công nghiệp Xanh là một cách tiếp cận toàn diện có tác động lan tỏa rộng rãi, bắt đầu và tập trung vào công nghiệp, nhưng có liên quan đến tất cả mọi mặt của xã hội. Sáng tạo và đổi mới kỹ thuật là động lực cho các ngành công nghiệp xanh của tương lai, sẽ là tác nhân xúc tiến việc làm xanh, tăng trưởng trong tương lai, và phát triển bền vững hơn.
Hiện nay tại Việt Nam, ngành công nghiệp môi trường có thể nói còn khá mới mẻ và đang tìm hướng phát triển phù hợp. Việt Nam đã có nhiều nỗ lực trong việc xây dựng khung khổ pháp lý phù hợp cho các hoạt động bảo vệ môi trường. Các quy định về đánh giá tác động môi trường, kiểm soát ô nhiễm, xử lý nước thải, quản lý chất thải rắn, chất thái nguy hại, kế hoạch xử lý triệt để cơ sở gây ô nhiễm môi trường nghiêm trọng … ngày càng rõ và cụ thể hơn. Công tác kiểm tra, thành tra, xử lý vi phạm pháp luật kèm theo các chế tài hành chính, hình sự và các biện pháp bổ sung khác cũng ngày càng được quan tâm hoan. Việc thực hiện nguyên tắc “người gây ô nhiễm phải trả” thông qua các quy định về thuế, phí, lệ phí môi trường như phí bảo vệ môi trường đối với nước thải, chất thải rắn, khai thác khoáng sản, phí xăng dầu,…, cơ chế ký quỹ phục hồi môi trường, cơ chế bồi thường thiệt hại cũng dần hình thành và đi vào thực hiện. Đến năm 2009, Chính phủ đã phê duyệt một đề án gọi là Đề án phát triển ngành công nghiệp môi trường Việt Nam đến năm 2015 và tầm nhìn đến năm 2025. Mục tiêu của đề án này là phát triển ngành công nghiệp môi trường thành ngành công nghiệp có khả năng cung cấp các công nghệ, thiết bị, dịch vụ sản phẩm môi trường nhằm xử lý, kiểm soát ô nhiễm, khắc phục suy thoái, hạn chế mức độ gia tăng ô nhiễm và cải thiện chất lượng môi trường.
Công nghiệp hóa, hiện đại hóa với môi trường là hai vấn đề có mối quan hệ hữu cơ với nhau. Nếu chúng ta chỉ chú ý đến công nghiệp hóa, hiện đại hóa để đem lại hiệu quả kinh tế thuần túy, mà quên vấn đề môi trường thì hiệu quả kinh tế thu được chắc gì đã đủ để giải quyết hậu quả do ô nhiễm môi trường gây ra. Chính vì vậy, dù ở bất kỳ cơ sở sản xuất nào, khu công nghiệp nào, thành thị hay nông thôn, cũng phải gắn liền với vấn đề môi trường xanh, sạch, đẹp để phát triển nền kinh tế-xã hội một cách bền vững. Chính sách nhất quán của Việt Nam là phát triển công nghiệp trên cơ sở phát triển bền vững, chuyển dần sang phát triển công nghiệp xanh nhằm bảo vệ môi trường, môi sinh và cân bằng sinh thái.
Tại Việt Nam, khái niệm kinh tế thị trường định hướng xã hội chủ nghĩa được chính thức sử dụng trong các văn kiện Đại hội IX của Đảng (tháng 4-2001); theo đó, “Đảng và Nhà nước ta chủ trương thực hiện nhất quán và lâu dài chính sách phát triển kinh tế hàng hóa nhiều thành phần vận động theo cơ chế thị trường có sự quản lý của Nhà nước theo định hướng xã hội chủ nghĩa, đó chính là nền kinh tế thị trường định hướng xã hội chủ nghĩa” . Đại hội lần thứ XII của Đảng (tháng 1-2016) tiếp tục bổ sung, phát triển: “Nền kinh tế thị trường định hướng xã hội chủ nghĩa Việt Nam là nền kinh tế vận hành đầy đủ, đồng bộ theo các quy luật của kinh tế thị trường, đồng thời bảo đảm định hướng xã hội chủ nghĩa phù hợp với từng giai đoạn phát triển của đất nước. Đó là nền kinh tế thị trường hiện đại và hội nhập quốc tế; có sự quản lý của Nhà nước pháp quyền xã hội chủ nghĩa” .
Thời kỳ đổi mới, công nghiệp hóa, hiện đại hóa được tiến hành trong nền kinh tế thị trường định hướng xã hội chủ nghĩa, nhiều thành phần. Công nghiệp hóa, hiện đại hóa không phải chỉ là việc của Nhà nước mà là sự nghiệp của toàn dân, của mọi thành phần kinh tế. Công nghiệp hóa, hiện đại hóa gắn với phát triển kinh tế thị trường không những khai thác có hiệu quả mọi nguồn lực trong nền kinh tế, mà còn sử dụng chúng có hiệu quả để đẩy nhanh quá trình công nghiệp hóa, hiện đại hóa đất nước. Bởi vì, khi đầu tư vào lĩnh vực nào, ở đâu, quy mô thế nào, công nghệ gì đều đòi hỏi phải tính toán, cân nhắc kỹ càng, hạn chế đầu tư tràn lan, sai mục đích, kém hiệu quả và lãng phí thất thoát.
Công nghiệp hóa, hiện đại hóa nền kinh tế ở nước ta hiện nay diễn ra trong bối cảnh toàn cầu hóa kinh tế, tất yếu phải hội nhập và mở rộng quan hệ kinh tế quốc tế. Hội nhập kinh tế quốc tế, mở rộng quan hệ kinh tế đối ngoại nhằm thu hút nguồn vốn đầu tư nước ngoài, thu hút công nghệ hiện đại, học hỏi kinh nghiệm quản lý tiên tiến của thế giới… sớm đưa nước ta ra khỏi tình trạng kém phát triển. Nói cách khác, đó là việc kết hợp sức mạnh dân tộc với sức mạnh thời đại để phát triển kinh tế nói chung và công nghiệp hóa, hiện đại hóa nói riêng được nhanh hơn, hiệu quả hơn.
Đại hội IX của Đảng đã đề ra chủ trương: “Chủ động hội nhập kinh tế quốc tế và khu vực theo tinh thần phát huy tối đa nội lực, nâng cao hiệu quả hợp tác quốc tế, bảo đảm độc lập tự chủ và định hướng xã hội chủ nghĩa, bảo vệ lợi ích dân tộc, an ninh quốc gia, giữ gìn bản sắc văn hóa dân tộc, bảo vệ môi trường” . Đại hội X của Đảng (năm 2006) tiến thêm một bước trong nhận thức và hành động hội nhập quốc tế; đề ra chủ trương: “Chủ động và tích cực hội nhập kinh tế quốc tế, đồng thời mở rộng hợp tác quốc tế trên các lĩnh vực khác. Việt Nam là bạn, đối tác tin cậy của các nước trong cộng đồng quốc tế, tham gia tích cực vào tiến trình hợp tác quốc tế và khu vực” . Đến Đại hội XI của Đảng, Việt Nam nhấn mạnh đến hội nhập quốc tế: “Thực hiện nhất quán đường lối đối ngoại độc lập, tự chủ, hòa bình, hợp tác và phát triển; đa phương hóa, đa dạng hóa quan hệ, chủ động và tích cực hội nhập quốc tế; là bạn, đối tác tin cậy và thành viên có trách nhiệm trong cộng đồng quốc tế; vì lợi ích quốc gia, dân tộc, vì một nước Việt Nam xã hội chủ nghĩa giàu mạnh” .
Với đường lối đối ngoại đa phương hóa, đa dạng hóa đã cũng cố môi trường hòa bình, ổn định, nâng cao vị thế, vai trò của Việt Nam trên trường quốc tế. Mặt khác, đã tạo điều kiện hội nhập với nền kinh tế thế giới góp phần phát triển kinh tế - xã hội và là điều kiện để chúng ta tiếp thu những thành tựu khoa học và công nghệ của nhân loại.
Sau 35 năm đổi mới, chúng ta đã giành được những thành tựu to lớn, có ý nghĩa lịch sử, đất nước thoát khỏi khủng hoảng kinh tế - xã hội, trở thành nước có thu nhập trung bình; hội nhập mạnh mẽ vào kinh tế khu vực và thế giới, tạo thế và lực mới cho nền kinh tế. Liên tiếp trong 4 năm, từ năm 2016 - 2019, Việt Nam đứng trong top 10 nước tăng trưởng cao nhất thế giới, là một trong 16 nền kinh tế mới nổi thành công nhất. Đặc biệt, trong năm 2020, trong khi phần lớn các nước có mức tăng trưởng âm hoặc đi vào trạng thái suy thoái do tác động của đại dịch COVID-19, kinh tế Việt Nam vẫn tăng trưởng 2,91%, góp phần làm cho GDP trong 5 năm (2016 - 2020) tăng trung bình 5,9%/năm, thuộc nhóm nước có tốc độ tăng trưởng cao nhất khu vực và thế giới. Quy mô nền kinh tế được nâng lên, nếu như năm 1989 mới đạt 6,3 tỷ USD thì đến năm 2020 đã đạt khoảng 343 tỷ USD.
Tuy vậy, trong quá trình phát triển kinh tế thị trường định hướng xã hội chủ nghĩa, vẫn còn nhiều hạn chế, yếu kém. Cụ thể là:
Thứ nhất, quá trình đổi mới nhận thức về kinh tế thị trường định hướng xã hội chủ nghĩa diễn ra chậm, nhất là nhận thức về bản chất và nguyên tắc vận hành của kinh tế thị trường định hướng xã hội chủ nghĩa, nên đã tạo ra sự thiếu đồng bộ, thiếu nhất quán và những bất cập trong quá trình hoàn thiện thể chế kinh tế thị trường.
Thứ hai, tăng trưởng kinh tế còn chậm, chưa bền vững, vẫn ở dưới mức tiềm năng, lực lượng sản xuất chưa được giải phóng triệt để, năng suất lao động thấp, khả năng cạnh tranh quốc tế chưa cao.
Thứ ba, việc phân bổ các nguồn lực cho phát triển còn dàn trải, lãng phí, chưa công bằng, chưa đem lại hiệu quả cao; vấn đề phân hóa giàu nghèo, bất bình đẳng xã hội, tệ nạn xã hội có chiều hướng gia tăng.
Đại hội XIII của Đảng nhấn mạnh: “Đổi mới mạnh mẽ và có hiệu quả mô hình tăng trưởng, cơ cấu lại nền kinh tế, đẩy mạnh công nghiệp hóa, hiện đại hóa trên nền tảng khoa học và công nghệ, đổi mới sáng tạo và nguồn nhân lực chất lượng cao. Xây dựng, hoàn thiện đồng bộ thế chế phát triển nền kinh tế thị trường định hướng xã hội chủ nghĩa đầy đủ, hiện đại, hội nhập”.
Thống nhất và nâng cao nhận thức về phát triển kinh tế thị trường định hướng xã hội chủ nghĩa. Đó là nền kinh tế thị trường hiện đại, hội nhập quốc tế, vận hành đầy đủ, đồng bộ theo các quy luật của kinh tế thị trường, có sự quản lý của Nhà nước pháp quyền xã hội chủ nghĩa, do Đảng Cộng sản Việt Nam lãnh đạo; bảo đảm định hướng xã hội chủ nghĩa vì mục tiêu “dân giàu, nước mạnh, dân chủ, công bằng, văn minh” phù hợp với từng giai đoạn phát triển của đất nước.
Tiếp tục hoàn thiện đồng bộ thể chế kinh tế thị trường định hướng xã hội chủ nghĩa, tập trung tháo gỡ các điểm nghẽn. Tập trung sửa đổi những quy định mâu thuẫn, chồng chéo, cản trở phát triển kinh tế. Đẩy mạnh cải cách thủ tục hành chính, phân cấp, phân quyền, ủy quyền gắn với tăng cường kỷ luật, kỷ cương, trách nhiệm cá nhân và nâng cao trách nhiệm phối hợp giữa các cấp, các ngành.
Để tăng trưởng kinh tế cần 5 yếu tố chủ yếu là: vốn, khoa học và công nghệ, con người, cơ cấu kinh tế, thể chế chính trị và quản lý nhà nước, trong đó con người là yếu tố quyết định. Để phát triển nguồn lực con người đáp ứng yêu cầu của công nghiệp hóa, hiện đại hóa cần đặc biệt chú ý đến phát triển giáo dục, đào tạo.
Quan điểm này, về cơ bản, đã phản ánh được phạm vi rộng lớn, lâu dài, phức tạp cũng như chỉ ra cốt lõi của quá trình công nghiệp hóa, hiện đại hóa của nước ta là cải biến lao động thủ công, lạc hậu thành lao động sử dụng kỹ thuật tiên tiến, hiện đại để đạt tới năng suất lao động xã hội cao. Trên cơ sở quan điểm này, Đảng ta chỉ rõ, sự nghiệp công nghiệp hóa, hiện đại hóa ở nước ta hiện nay là một cuộc cách mạng toàn diện và sâu sắc trong tất cả các lĩnh vực của đời sống kinh tế-xã hội, nhằm mục tiêu xây dựng nước ta thành một nước công nghiệp có cơ sở vật chất kỹ thuật hiện đại.
Con người vừa là mục tiêu vừa là động lực của quá trình công nghiệp hóa, hiện đại hóa. Động lực của sự nghiệp công nghiệp hóa, hiện đại hóa được hiểu là những gì thúc đẩy quá trình này vận động và phát triển. Khi nói con người là động lực của quá trình công nghiệp hóa, hiện đại hóa là chủ yếu nói đến sức mạnh, năng lực sáng tạo to lớn của con người trong việc thực hiện công nghiệp hóa, hiện đại hóa và thúc đẩy quá trình này phát triển. Nếu con người là động lực, là phương tiện để tiến hành công nghiệp hóa, hiện đại hóa, thì những thành quả do công nghiệp hóa, hiện đại hóa tạo ra phải nhằm phục vụ cho nhu cầu của con người. Nói cách khác, con người có quyển thụ hưởng những thành quả do chính tài năng và sức sáng tạo của mình tạo ra. 
Để phát huy tính tích cực, tự giác, sáng tạo của con người trong mọi hoạt động thì nhất thiết xây dựng và thực hiện một cơ chế dân chủ, trong đó ngày càng quan tâm đến dân chủ trực tiếp nhằm hiện thực hóa những giá trị dân chủ xã hội chủ nghĩa trong đời sống xã hội. Việc phát huy dân chủ không chỉ tạo cơ hội thuận lợi cho tất cả mọi người có thể cống hiến năng lực của mình cho xã hội, mà còn có ý nghĩa tạo điều kiện cho việc thực hiện cơ chế phản biện xã hội với tinh thần lành mạnh, xây dựng và phát huy trách nhiệm công dân. Tuy nhiên, thực hiện dân chủ phải đi đôi với tăng cường kỷ cương pháp luật, pháp chế xã hội chủ nghĩa; khắc phục mọi biểu hiện dân chủ cực đoan hoặc lợi dụng “dân chủ” để chia rẽ khối đại đoàn kết dân tộc, làm rạn nứt sự đồng thuận xã hội.
Với những quan điểm của Đảng ta về phát huy nhân tố con người trong suốt chặng đường phát triển đất nước đã phát huy hiệu quả và đạt được nhiều thành tựu to lớn: Trình độ dân trí ngày càng cao, kinh tế cũng ngày càng khởi sắc và trên đà phát triển ổn định, đời sống của người dân được cải biến rõ nét, con người được tạo điều kiện phát triển toàn diện, nhân dân ngày càng tin tưởng vào sự lãnh đạo của Đảng.
Trong xã hội hiện nay, tất yếu còn một bộ phận nhân dân đời sống còn nhiều khó khăn vì những nguyên nhân khách quan và chủ quan. Với truyền thống đoàn kết, nhân nghĩa tương thân, tương ái của con người Việt Nam, Đảng ta khẳng định tiếp tục sửa đổi, hoàn thiện hệ thống bảo hiểm xã hội, bảo hiểm y tế, … giúp đỡ mọi thành viên trong xã hội, nhất là các nhóm yếu thế, dễ bị tổn thương để vượt qua khó khăn trong đời sống.
Khoa học và công nghệ có vai trò quyết định đến tăng năng suất lao động, giảm chi phí sản xuất, nâng cao lợi thế cạnh tranh và tốc độ phát triển kinh tế nói chung. Nước ta tiến lên chủ nghĩa xã hội từ một nền kinh tế kém phát triển và tiềm lực khoa học, công nghệ còn ở trình độ thấp. Muốn đẩy nhanh quá trình công nghiệp hóa, hiện đại hóa gắn với kinh tế tri thức thì phát triển khoa học và công nghệ là yêu cầu tất yếu và bức xúc. 
Thực tế trong sự nghiệp công nghiệp hóa, hiện đại hóa hiện nay, khoa học và công nghệ có vai trò và tầm quan trọng vô cùng to lớn và được cụ thể hóa trong các văn kiện của Đảng và Nhà nước, trong chỉ đạo, định hướng của các lãnh đạo Đảng và Nhà nước. Trong Báo cáo tổng kết 20 năm thực hiện Cương lĩnh xây dựng đất nước trong thời kỳ quá độ lên Chủ nghĩa xã hội (1991-2011), Đảng ta đã chỉ rõ: “Khoa học và công nghệ cùng với giáo dục và đào tạo phải được coi là quốc sách hàng đầu. Khoa học và công nghệ giữ vai trò then chốt trong phát triển lực lượng sản xuất, phân bố lại lực lượng lao động; nâng cao trình độ quản lý, tăng sức cạnh tranh của sản phẩm hàng hoá, bảo đảm an sinh xã hội và bảo vệ môi trường - sinh thái, bảo đảm chất lượng và tốc độ phát triển kinh tế; góp phần tích cực trong đấu tranh bảo vệ chủ quyền của đất nước…”.
Đặc biệt, Hội nghị lần thứ 6 Ban Chấp hành Trung ương Đảng khóa XI đã ban hành Nghị quyết về “Phát triển khoa học và công nghệ phục vụ sự nghiệp công nghiệp hóa, hiện đại hóa trong điều kiện thị trường định hướng xã hội chủ nghĩa và hội nhập quốc tế” . Nghị quyết nêu rõ: “Phát triển khoa học và ứng dụng khoa học và công nghệ là quốc sách hàng đầu, là một trong những động lực quan trọng nhất để phát triển kinh tế- xã hội và bảo vệ Tổ quốc”. Đồng thời “ưu tiên và tập trung mọi nguồn nhân lực quốc gia cho phát triển khoa học và công nghệ”.
Tiếp tục cụ thể hóa nhận thức này, Điều 62, Hiến pháp năm 2013 đã khẳng định: “Phát triển khoa học và công nghệ là quốc sách hàng đầu; giữ vai trò then chốt trong sự nghiệp phát triển kinh tế - xã hội của đất nước”. Theo đó, đến nay Quốc hội đã ban hành 8 đạo luật chuyên ngành, đặc biệt là Luật Khoa học và công nghệ được Quốc hội khóa XIII thông qua; Chiến lược khoa học công nghệ giai đoạn 2011-2020 và các văn bản pháp lý khác đã tạo điều kiện thúc đẩy môi trường sáng tạo, ứng dụng và chuyển giao công nghệ phát triển.
Khoa học và công nghệ còn có tác dụng to lớn trong việc truyền bá hệ tư tưởng chính trị xã hội chủ nghĩa, xây dựng nền văn hóa tiên tiến đậm đà bản sắc dân tộc, xây dựng lối sống, đạo đức và nhân cách mới của toàn bộ xã hội. Sự nghiệp công nghiệp hóa, hiện đại hóa đất nước không phải chỉ là quá trình đổi mới về Khoa học và công nghệ, hiện đại hóa, thị trường hóa nền sản xuất xã hội mà còn là quá trình chuyển đổi về tâm lý, phong tục tập quán, lối sống thích ứng với nhịp độ và tốc độ của xã hội công nghiệp và hội nhập kinh tế quốc tế. Đại hội đại biểu lần thứ XII của Đảng đã đề ra mục tiêu đến năm 2020, khoa học và công nghệ Việt Nam đạt trình độ phát triển của nhóm các nước dẫn đầu ASEAN và đồng thời khẳng định vai trò của khoa học công nghệ: “Phát triển mạnh mẽ khoa học và công nghệ, làm cho khoa học và công nghệ thực sự là quốc sách hàng đầu, là động lực quan trọng nhất để phát triển lực lượng sản xuất hiện đại, kinh tế tri thức, nâng cao năng suất, chất lượng, hiệu quả và sức cạnh tranh của nền kinh tế; bảo vệ môi trường, bảo đảm quốc phòng, an ninh. “. 
Nước ta tiến lên Chủ nghĩa Xã hội từ một nền kinh tế kém phát triển, lạc hậu và tiềm lực khoa học, công nghệ còn ở trình độ thấp. Muốn đẩy nhanh quá trình công nghiệp hóa, hiện đại hóa gắn với tri thức thì phát triển khoa học và công nghệ là yêu cầu tất yếu. Khoa học và công nghệ cùng với giáo dục đào tạo được xem là quốc sách hàng đầu, là nền tảng và động lực của công nghiệp hóa, hiện đại hóa. Và tại Đại hội XIII vừa qua Đảng lại nhấn mạnh: "Thúc đẩy đổi mới sáng tạo, ứng dụng mạnh mẽ khoa học và công nghệ, nhất là những thành tựu của cuộc cách mạng công nghiệp lần thứ tư, tạo động lực mạnh mẽ cho phát triển nhanh và bền vững". 
Trải qua vài thập kỷ xây dựng và phát triển, đến nay, chúng ta đã có một tiềm lực khoa học công nghệ (Khoa học và công nghệ) đáng kể, lực lượng cán bộ Khoa học và công nghệ tương đối đông đảo. Thực tế cho thấy, đội ngũ này có khả năng tiếp thu tương đối nhanh và làm chủ được tri thức, công nghệ hiện đại trên một số ngành và lĩnh vực. Cùng với đội ngũ cán bộ Khoa học và công nghệ đông đảo nói trên, chúng ta đã xây dựng được một mạng lưới tổ chức Khoa học và công nghệ thuộc mọi thành phần kinh tế, trong đó có gần phân nửa tổ chức ngoài nhà nước . Cơ sở hạ tầng kỹ thuật của các viện, trung tâm nghiên cứu, các phòng thí nghiệm, các trung tâm thông tin Khoa học và công nghệ, thư viện cũng được tăng cường và nâng cấp.
Cùng với quá trình đổi mới cơ chế kinh tế, trong những năm qua, cơ chế quản lý trong lĩnh vực này từng bước được đổi mới theo hướng mở rộng liên kết giữa nghiên cứu với sản xuất, kinh doanh. Nhờ đó, đã nâng cao hiệu quả hoạt động Khoa học và công nghệ, góp phần nâng cao năng suất, đóng góp đáng kể cho tăng trưởng và sức cạnh tranh của nền kinh tế, bảo đảm quốc phòng, an ninh, góp phần phòng chống thiên tai, bảo vệ môi trường, ứng phó với biến đổi khí hậu và phát triển bền vững đất nước. Có thể điểm một số thành tựu chủ yếu trong các lĩnh vực hoạt động Khoa học và công nghệ sau đây.
Đáng chú ý là khoa học tự nhiên đã có kết quả đáng kể về nghiên cứu cơ bản, một số lĩnh vực có thế mạnh như toán, vật lý lý thuyết đạt thứ hạng cao trong ASEAN. Khoa học ứng dụng có khả năng thiết kế, chế tạo thành công nhiều công nghệ, thiết bị nội địa theo tiêu chuẩn quốc tế, có năng lực hấp thụ, làm chủ công nghệ mới, công nghệ cao trong một số ngành như: điện, điện tử, dầu khí, năng lượng, cơ khí lắp ráp ô tô, xe máy và hàng điện tử đa dùng, đóng tàu, xây dựng, y tế, thông tin và truyền thông... Chỉ số đổi mới sáng tạo của Việt Nam năm 2017 tăng 12 bậc, năm 2018 tăng tiếp 2 bậc, xếp thứ 45/126 quốc gia. Trong đó, nhóm chỉ số về tri thức - công nghệ của Việt Nam có thứ hạng cao, xếp thứ 28. Đặc biệt, năm 2020 Việt Nam xếp thứ 42/131 quốc gia và nền kinh tế, còn năm 2019 đứng thứ 42/129 quốc gia. Với thứ hạng này, Việt Nam giữ vị trí số một trong nhóm 29 quốc gia có cùng mức thu nhập và đứng thứ 3 khu vực Đông Nam Á.
Khoa học xã hội và nhân văn trong thời kỳ đổi mới đã có những đóng góp tích cực vào việc phát triển lý luận và tổng kết thực tiễn xây dựng chủ nghĩa xã hội ở nước ta. Nhiều kết quả nghiên cứu đã đóng góp cho việc chuẩn bị các văn kiện của Đảng; xây dựng chiến lược phát triển kinh tế - xã hội của đất nước qua các thời kỳ; xây dựng và hoàn thiện hệ thống pháp luật. 
Trong nông nghiệp, khoa học công nghệ đóng góp trên 30% giá trị gia tăng trong sản xuất nông nghiệp, 38% trong sản xuất giống cây trồng, vật nuôi. Mức độ tổn thất của nông sản đã giảm đáng kể (lúa gạo còn dưới 10%,...). Mức độ cơ giới hóa ở khâu làm đất đối với các loại cây hàng năm (lúa, mía, ngô, rau màu) đạt khoảng 94%; khâu thu hoạch lúa đạt 50% (các tỉnh đồng bằng đạt 90%). Kim ngạch xuất khẩu nông sản năm 2018 ước đạt 40,02 tỷ USD; tiếp tục duy trì 10 nhóm mặt hàng có kim ngạch xuất khẩu trên 17 tỷ USD, trong đó có 05 mặt hàng có kim ngạch trên 3 tỷ USD (gỗ và sản phẩm gỗ, tôm, rau quả, cà phê, hạt điều). Đã xuất hiện nhiều mô hình nông nghiệp ứng dụng công nghệ cao, công nghiệp công nghệ cao với sự đầu tư mạnh của các doanh nghiệp, tập đoàn lớn trong nước và nước ngoài.
Trong công nghiệp, xây dựng, giao thông, các nhiệm vụ khoa học công nghệ chú trọng vào hỗ trợ phát triển sản phẩm của doanh nghiệp theo chuỗi giá trị để tạo ra sản phẩm, hàng hóa mang nhãn hiệu Việt Nam, có giá trị gia tăng và tính cạnh tranh cao trên thị trường; nâng cao năng suất, chất lượng sản phẩm, hàng hóa ngành công nghiệp. Đã làm chủ công nghệ thiết kế, thi công các nhà máy thủy điện lớn, phát triển nguồn điện, lưới điện, hệ thống điều khiển - điều độ - thông tin - viễn thông điện lực, công trình ngầm, nhà cao tầng, cầu dây văng, đường cao tốc đạt chuẩn quốc tế.
Khoa học và công nghệ ngày càng khẳng định vai trò bệ đỡ cho nhiều ngành nghề, lĩnh vực...Thực tiễn được chứng minh qua đại dịch COVID-19. Ngay từ những ngày đầu bùng phát dịch, thực hiện chỉ đạo của Ban Bí thư, Chính phủ, ngành khoa học và công nghệ đã tích cực huy động các chuyên gia, nhà khoa học hàng đầu trong và ngoài nước, các tổ chức, doanh nghiệp khẩn trương triển khai các nhiệm vụ khoa học và công nghệ  phục vụ phòng, chống dịch theo phương châm “chống dịch như chống giặc” và đã đạt được những kết quả bước đầu đáng ghi nhận như việc sản xuất thành công bộ KIT phát hiện virus SARS-CoV-2 được công nhận về chất lượng để sử dụng rộng rãi trên toàn cầu. Các tổ chức khoa học và công nghệ của nước ta cũng đã nuôi cấy và phân lập thành công virus SARS-CoV-2, tạo tiền đề cho việc nghiên cứu sâu hơn về virus, đồng thời cung cấp vật liệu và hỗ trợ tích cực cho nghiên cứu chế tạo bộ KIT, kháng thể đơn dòng và vắc xin...Việt Nam cũng đã sản xuất và đưa vào thử nghiệm thành công nhiều loại robot phục vụ chăm sóc y tế cho người nhiễm bệnh.
Bên cạnh những kết quả quan trọng nói trên, đến nay một số chủ trương lớn về khoa học và công nghệ được nêu trong các văn bản quan trọng của Đảng, Nhà nước chưa đạt được yêu cầu, như đến năm 2020, khoa học và công nghệ Việt Nam đạt trình độ phát triển của nhóm các nước dẫn đầu ASEAN; làm chủ, ứng dụng và phát triển các công nghệ có ảnh hưởng quyết định đến tốc độ và chất lượng tăng trưởng của nền kinh tế, tạo ra các sản phẩm mới có tính cạnh tranh cao. Trình độ khoa học và công nghệ Việt Nam vẫn còn một khoảng cách khá xa so với các nước phát triển và ngay cả với nhiều nước trong khu vực, chưa tạo ra được năng lực Khoa học và công nghệ cần thiết để đáp ứng những yêu cầu mới của công nghiệp hóa, hiện đại hóa đất nước. Năng suất lao động trong nhiều ngành kinh tế của Việt Nam còn rất thấp. Tác động của khoa học và công nghệ đối với đổi mới mô hình tăng trưởng, cơ cấu lại nền kinh tế, nâng cao sức cạnh tranh và tham gia chuỗi giá trị toàn cầu còn hạn chế. Các xếp hạng quốc tế về lĩnh vực khoa học công nghệ và đổi mới sáng tạo của chúng ta đều ở mức thấp, thí dụ: Chỉ số chính phủ điện tử xếp hạng 88/193 quốc gia (Liên hiệp quốc năm 2018); chỉ số công nghệ và đổi mới sáng tạo xếp thứ 90/100 và chỉ số vốn con người xếp thứ 70/100, chỉ số về nguồn lực bền vững xếp thứ 87/100 (Diễn đàn Kinh tế thế giới năm 2018).
Đại hội toàn quốc lần thứ XIII của Đảng vừa qua đã chỉ rõ con đường để chúng ta đẩy mạnh xây dựng tiềm lực và đổi mới cơ chế quản lý tạo tiền đề cho sự hình thành bước đột phá tư duy trong đẩy mạnh công nghiệp hóa, hiện đại hóa với các giải pháp chủ yếu sau.
Thứ nhất, thúc đẩy chuyển đổi tư duy từ phát triển nền công nghiệp phụ thuộc, gia công, lắp ráp sang chủ động sáng tạo, vươn lên, làm chủ công nghệ. Thực tế cho thấy, công nghiệp, công nghiệp hỗ trợ nước ta còn phát triển chậm, trình độ hạn chế, nhiều nguyên liệu đầu vào quan trọng chưa sản xuất được nên phụ thuộc lớn vào nhập khẩu; việc tham gia trong chuỗi giá trị toàn cầu ở nhiều sản phẩm vẫn chỉ ở công đoạn cuối cùng nên giá trị gia tăng không lớn. Đại hội XIII đánh giá: “Mô hình tăng trưởng chưa dựa trên nền tảng khoa học - công nghệ, đổi mới sáng tạo, tính tự chủ của nền kinh tế còn thấp, vẫn còn phụ thuộc vào bên ngoài”. Do vậy, Đại hội XIII chủ trương: “Thúc đẩy khởi nghiệp sáng tạo, phát triển các ngành, lĩnh vực, các doanh nghiệp trên nền tảng ứng dụng mạnh mẽ các thành tựu của khoa học - công nghệ, nhất là cuộc Cách mạng công nghiệp lần thứ tư”.
Thứ hai, chuyển đổi mạnh mẽ nền kinh tế công nghiệp sang nền kinh tế số, dựa trên nền tảng tri thức. Đây là điểm mới khác biệt so với các đại hội trước, là điểm nhấn của Đại hội XIII. Ngày nay, với sự chuyển động mạnh mẽ của nền kinh tế toàn cầu trong kỷ nguyên Cách mạng công nghiệp lần thứ tư, Việt Nam bước vào giai đoạn đổi mới toàn diện, chủ động tham gia vào quá trình hội nhập quốc tế ngày càng sâu rộng. Đại hội XIII chủ trương “đẩy mạnh chuyển đổi số quốc gia, phát triển kinh tế số dựa trên nền tảng khoa học - công nghệ, đổi mới sáng tạo”, trong đó con người hay tài nguyên trí tuệ là nền tảng cốt lõi, doanh nghiệp phải là trung tâm nghiên cứu phát triển, ứng dụng và chuyển giao công nghệ, ứng dụng công nghệ số. 
Thứ ba, thúc đẩy chuyển đổi quá trình công nghiệp hóa, hiện đại hóa sang giai đoạn mới: thâm dụng tri thức và đột phá đổi mới sáng tạo. Đại hội XIII chủ trương “chuyển mạnh nền kinh tế sang mô hình tăng trưởng dựa trên năng suất, tiến bộ khoa học - công nghệ, đổi mới sáng tạo, nhân lực chất lượng cao”. Chủ trương này không chỉ nhấn mạnh phát triển khoa học - công nghệ, mà còn đề cao yêu cầu đổi mới sáng tạo như một định hướng trung tâm, xuyên suốt trong xu thế Cách mạng công nghiệp lần thứ tư đang tăng tốc. Hiện nay, nền kinh tế Việt Nam chưa đạt trình độ công nghiệp hóa, hiện đại hóa như các nước tiên tiến trên thế giới, nhưng cuộc Cách mạng công nghiệp lần thứ tư đã mở ra cơ hội để nước ta thay vì “đi sau” thì có thể nỗ lực để “đi cùng”, một số lĩnh vực mũi nhọn, có thế mạnh, có thể phấn đấu “đi trước, vượt trước”. 
Vậy nên, trong sự nghiệp công nghiệp hóa, hiện đại hóa hiện nay khoa học và công nghệ chiếm một vị trí đặc biệt quan trọng. Trong thời đại ngày nay, sự nghiệp công nghiệp hóa, hiện đại hóa ở nước ta là nhằm xây dựng cơ sở vật chất kỹ thuật cho Chủ nghĩa Xã hội. Đó là nhiệm vụ trung tâm trong suốt thời kỳ quá độ tiến lên Chủ nghĩa Xã hội ở nước ta. 
Mặt khác, việc chú trọng phát triển kinh tế mà không quan tâm đến giải quyết những vấn đề bình đẳng xã hội sẽ dẫn đến tình trạng gia tăng khoảng cách giàu nghèo, bất công xã hội, tạo nguy cơ gây mất ổn định chính trị -xã hội, dẫn đến khủng hoảng thể chế. Do vậy phát triển công bằng, bền vững ngày càng trở thành mục tiêu quan trọng, tổng quan xuyên suốt mà nhiều quốc gia trên thế giới phải quan tâm hướng tới.
Khi phát triển nhanh, hiệu quả và bền vững, đất nước mới có khả năng xóa đói, giảm nghèo, nâng cao đời sống vật chất, tinh thần của nhân dân, phát triển văn hóa, giáo dục, y tế, rút ngắn khoảng cách chênh lệch giữa các vùng… Mục tiêu đó hướng đến mục tiêu quan trọng là xã hội tiến bộ và công bằng xã hội, mọi người dân đều được hưởng thành quả của sự phát triển.
Quan niệm phát triển bền vững xuất hiện từ những năm đầu thập kỷ 80 thế kỷ XX, được phổ biến rộng rãi vào cuối những năm 1980, với hàm ý tổng quát là sự phát triển đáp ứng được những nhu cầu hiện tại nhưng không trở ngại cho việc đáp ứng nhu cầu của các thế hệ mai sau. Hội nghị thượng đỉnh trái đất về môi trường và phát triển họp tại Rio de Janeiro (Brazil) năm 1992 đã  thể hiện tinh thần toàn cầu cao độ và là bước khởi đầu cho khái niệm phát triển bền vững. 
Thế giới của chúng ta hiện nay hiện đang đối mặt với vô vàn thách thức về môi trường, như môi trường ngày càng bị ô nhiễm nghiêm trọng, tình trạng biến đổi khí hậu, nước biển dâng cùng với nguy cơ về khủng hoảng lương thực, khủng hoảng năng lượng, khủng hoảng nguồn nước, khủng hoảng tài chính kinh tế. Trong bối cảnh đó, nhiều quốc gia muốn tăng trưởng kinh tế nhanh, song thiếu chú ý tới tác động đối với môi trường sinh thái, do vậy vừa làm cạn kiệt nguồn tài nguyên thiên nhiên vừa phải gánh chịu những hậu quả của một sự phát triển tàn phá môi trường. Do đó, sự phát triển nhanh, hiệu quả và bền vững, có quan hệ chặt chẽ với bảo vệ môi trường tự nhiên và bảo tồn đa dạng sinh học.  
Phát triển nhanh gắn với phát triển bền vững là phát triển kinh tế ổn định gắn với đảm bảo an sinh xã hội và bảo vệ môi trường sinh thái, vừa bao hàm cả phát triển theo chiều rộng và phát triển theo chiều sâu; đồng thời, thể hiện tốc độ và chất lượng của mô hình tăng trưởng kinh tế nước ta. Phát triển bền vững còn có thể hiểu là sự phát triển mà thế hệ hôm nay không làm tổn hại đến thế hệ ngày mai. 
Ở Việt Nam, xuất phát điểm từ một nền kinh tế sản xuất nhỏ lạc hậu, bị tàn phá nặng nề bởi sau hơn 30 năm chiến tranh, tụt hậu xa với khu vực và thế giới, đời sống nhân dân vô cùng khó khăn. Phù hợp với xu thế phát triển của thời đại và điều kiện cụ thể của đất nước, Đảng Cộng sản Việt Nam trong quá trình đổi mới đã lựa chọn chiến lược phát triển đất nước theo hướng nhanh và bền vững để thực hiện mục tiêu “ Dân giàu, nước mạnh, công bằng, dân chủ, văn minh”. 
Công cuộc đổi mới được Đảng cộng sản Việt Nam khởi xướng và lãnh đạo từ Đại hội VI năm 1986 đã mang lại những thành tựu to lớn, làm biến đổi sâu sắc đất nước trên các lĩnh vực như kinh tế, văn hóa, xã hội, con người. Trong đó phải kể đến những thành tựu quan trọng về tăng trưởng kinh tế mạnh mẽ đi đôi và gắn kết với việc đẩy nhanh tiến bộ và thực hiện công bằng xã hội, phát triển bền vững. Qua 35 năm đổi mới, quan điểm về phát triển công bằng, bền vững được Đảng cộng sản Việt Nam phát triển, không ngừng bổ sung và hoàn thiện qua các Đại hội Đảng.
Nếu tại Đại hội VI của Đảng đưa ra quan điểm “ổn định và phát triển gắn liền với nhau trong quá trình vận động tiến lên, ổn định để phát triển và có phát triển mới ổn định được”   thì đến Đại hội VII ( năm 1991) của Đảng Cộng sản Việt Nam thông qua Chiến lược ổn định và phát triển kinh tế-xã hội đến năm 2020 đã xác định: “Tăng trưởng kinh tế nhanh, hiệu quả cao và bền vững”; “ Tăng trưởng kinh tế, gắn liền với tiến bộ và công bằng xã hội, phát triển văn hóa và bảo vệ môi trường” .
Đại hội VIII của Đảng nhấn mạnh bài học: “ Tăng trưởng kinh tế gắn liền với tiến bộ và công bằng xã hội, giữ gìn và phát huy bản sắc văn hóa dân tộc, bảo vệ môi trường sinh thái” . Đại hội X (năm 2006) tiếp tục khẳng định quan điểm: “Phát triển nhanh, hiệu quả và bền vững, tăng trưởng kinh tế đi đôi với thực hiện tiến bộ, công bằng xã hội và bảo vệ môi trường” .
Đại hội XI (năm 2011) chỉ rõ: phát triển nhanh gắn với phát triển bền vững, phát triển bền vững là yêu cầu xuyên suốt trong Chiến lược phát triển kinh tế - xã hội đến năm 2020 và rút ra bài học: “ Phải thực sự coi trọng chất lượng, hiệu quả tăng trưởng và phát triển bền vững. Tăng cường huy động, gắn với sử dụng có hiệu quả các nguồn lực trong và ngoài nước”; “ Tăng trưởng kinh tế phải đi đôi với thực hiện tiến bộ và công bằng xã  hội và bảo vệ môi trường” .
Đại hội XII của Đảng (năm 2016) đã tổng kết 30 năm đổi mới ở Việt Nam  trên cơ sở kết quả đạt được và những vấn đề đặt ra, đã có bước phát triển mới về nhận thức với sự khẳng định: “ Bảo đảm phát triển nhanh, bền vững trên cơ sở bền vững kinh tế vĩ mô và không ngừng nâng cao năng suất, chất lượng, hiệu quả và sức cạnh tranh. Phát triển hài hòa giữa chiều rộng và chiều sâu, chú trọng phát triển chiều sâu; phát triển kinh tế tri thức, kinh tế xanh. Phát triển kinh tế, gắn kết chặt chẽ với phát triển văn hóa, xã hội, bảo vệ môi trường, chủ động ứng phó với biến đổi khí hậu. Bảo đảm quốc phòng, an ninh và giữ vững hòa bình, ổn định để xây dựng đất nước” . “Phát triển nhanh và bền vững (hướng tới các mục tiêu phát triển bền vững đến năm 2030 của Liên hợp quốc); giải quyết hài hòa giữa tăng trưởng kinh tế với phát triển văn hóa, thực hiện tiến bộ và công bằng xã hội, bảo vệ môi trường” .
Cụ thể hóa quan điểm của Đại hội XII, Hội nghị Trung ương 4 khóa XII  về đổi mới mô hình tăng trưởng đã xác định “đổi mới mô hình tăng trưởng để nâng cao chất lượng tăng trưởng, năng suất lao động và sức cạnh tranh của nền kinh tế, phát triển nhanh và bền vững toàn diện cả về kinh tế, xã hội và môi trường. Không thúc đẩy tăng trưởng bằng mọi giá, gây tác động xấu đến ổn định kinh tế vĩ mô, bảo đảm quốc phòng an ninh, tiến bộ công bằng xã hội, giữ gìn môi trường sinh thái.
Văn kiện Đại hội XIII trong bối cảnh mới đã chỉ rõ:“Chiến lược phát triển tổng thể của đất nước ta là đẩy mạnh toàn diện, đồng bộ công cuộc đổi mới, phát triển nhanh và bền vững đất nước; bảo đảm gắn kết chặt chẽ và triển khai đồng bộ các nhiệm vụ, trong đó: Phát triển kinh tế - xã hội là nhiệm vụ trung tâm; xây dựng đảng là then chốt; phát triển văn hóa là nền tảng tinh thần; đảm bảo quốc phòng, an ninh là trọng yếu, thường xuyên” .
Triển khai chủ trương chính sách của Đảng Cộng sản Việt Nam về phát triển nhanh và bền vững, qua 35 năm đổi mới, Việt Nam đã đạt được những kết quả quan trọng về phát triển kinh tế- xã hội và hội nhập quốc tế, thực hiện tốt mục tiêu Thiên niên kỷ, mục tiêu phát triển bền vững đến năm 2030 của Liên hiệp quốc.
Năm 2020, theo đánh giá của các tổ chức quốc tế, quy mô nền kinh tế Việt Nam nằm trong tốp 40 nền kinh tế lớn nhất thế giới và đứng thứ 4 trong ASEAN. GDP bình quân đầu người năm 2020 đạt trên 3.500 USD (tính theo sức mua tương đương đạt khoảng 10.000 USD). Mặc dù chịu tác động nặng nề của đại dịch Covid-19, song Việt Nam nằm trong tốp 10 quốc gia tăng trưởng cao nhất thế giới (Là một trong 16 nền kinh tế mới nổi thành công nhất thế giới). Việt nam là nền kinh tế mở với độ mở cao của thế giới khoảng 200% GDP. Hiện nay về đối ngoại, Việt Nam có quan hệ chính thức với 189/193 quốc gia; có quan hệ kinh tế, thương mại, đầu tư với trên 224 nước và vùng lãnh thổ; nước ta đã ký 15 Hiệp định FTA, có 16 đối tác chiến lược.
Trong giai đoạn phát triển mới, dưới tác động của quá trình toàn cầu hóa và cuộc Cách mạng công nghiệp lần thứ tư phát triển rất nhanh, văn kiện Đại hội XIII của Đảng đã chỉ rõ những nội dung mới: “Phát triển nhanh và bền vững dựa chủ yếu vào khoa học công nghệ, đổi mới sáng tạo và chuyển đổi số. Phải đổi mới tư duy và hành động, chủ động nắm bắt kịp thời, tận dụng hiệu quả các cơ hội của Cuộc cách mạng công nghiệp lần thứ tư gắn với quá trình hội nhập quốc tế để cơ cấu lại nền kinh tế, phát triển kinh tế số, xã hội số, coi đây là nhân tố quyết định để nâng cao năng suất lao động, chất lượng, hiệu quả và sức cạnh tranh”. Phát huy tối đa lợi thế của các vùng, miền; phát triển hài hòa giữa kinh tế với văn hóa, xã hội, bảo vệ môi trường và thích ứng với biến đổi khí hậu; quan tâm, tạo điều kiện thuận lợi cho các đối tượng chính sách, người có công, người nghèo, người yếu thế, đồng bào dân tộc thiểu số”. 
Tiến bộ xã hội là sự vận động phát triển của xã hội con người từ thấp lên cao; là sự phát triển của trình độ sản xuất, lực lượng sản xuất, quan hệ sản xuất. Sự tiến bộ xã hội còn thể hiện ở sự phát triển ngày càng cao hơn của cơ sở hạ tầng, kiến trúc thượng tầng và ý thức xã hội.
Công bằng xã hội là bảo đảm sự “ngang nhau” giữa người với người trong mối quan hệ giữa cống hiến với hưởng thụ, quyền lợi và nghĩa vụ, vinh dự với trách nhiệm. Thực tế phát triển của lịch sử nhân loại, công bằng xã hội là vấn đề mang tính lịch sử, được quy định bởi chế độ xã hội cụ thể.
Vì vậy, tăng trưởng kinh tế và tiến bộ, công bằng xã hội có mối quan hệ biện chứng với nhau, vừa là tiền đề, vừa là điều kiện của nhau. Tăng trưởng kinh tế là điều kiện để thực hiện tiến bộ, công bằng xã hội; tiến bộ, công bằng xã hội là nhân tố động lực để có tăng trưởng kinh tế cao và bền vững; tiến bộ, công bằng xã hội là biểu hiện của tăng trưởng kinh tế. Như vậy, tăng trưởng kinh tế và tiến bộ, công bằng xã hội không phải là những yếu tố đối lập mà có quan hệ nhân quả với nhau.
Định hướng xã hội chủ nghĩa với mục tiêu: “Dân giàu, nước mạnh, dân chủ, công bằng, văn minh” mà Đảng và nhân dân ta xác định và đang nỗ lực phấn đấu thực hiện cũng thể hiện tinh thần đó. Thực chất của tiến bộ và công bằng xã hội là giải quyết hài hòa giữa sự phát triển kinh tế với thực hiện các chính sách xã hội, đem lại cuộc sống hòa bình, độc lập, tự do, ấm no, hạnh phúc cho nhân dân. Phát triển kinh tế thị trường định hướng xã hội chủ nghĩa gắn với thực hiện tiến bộ và công bằng xã hội còn thể hiện tính ưu việt, nhân văn của chế độ xã hội chủ nghĩa hơn hẳn những chế độ xã hội trước, đồng thời phát huy tính năng động, khắc phục những bất cập cố hữu của cơ chế thị trường. 
Nhận thức sâu sắc về vai trò, ý nghĩa của tiến bộ xã hội, trong quá trình lãnh đạo cách mạng cũng như trong sự nghiệp đổi mới đất nước, Đảng ta xác định: “Thực hiện tiến bộ và công bằng xã hội ngay trong từng bước và từng chính sách phát triển; tăng trưởng kinh tế đi đôi với phát triển văn hóa, y tế, giáo dục…, giải quyết tốt vấn đề xã hội vì mục tiêu phát triển con người”. Đại hội XII của Đảng nêu rõ: “Gắn kết chặt chẽ, hài hòa giữa phát triển kinh tế với phát triển văn hóa và thực hiện tiến bộ, công bằng xã hội, nâng cao đời sống của nhân dân” . Đại hội XIII của Đảng xác định định hướng phát triển tiếp tục nắm vững và xử lý tốt mối quan hệ lớn “giữa tăng trưởng kinh tế và phát triển văn hóa, thực hiện tiến bộ, công bằng xã hội, bảo vệ môi trường” .
Các chính sách về phát triển văn hóa đậm đà bản sắc dân tộc và đang xây dựng 1 đời sống văn hóa phong phú, hiện đại như ngày nay. Công bằng xã hội ngày càng được thực hiện tốt hơn khi ta đang dần loại bỏ tư tưởng bất bình đẳng về giới tính, tôn giáo.
Hơn 35 năm đổi mới, cùng với phát triển kinh tế thị trường định hướng xã hội chủ nghĩa và hội nhập quốc tế, việc thực hiện tiến bộ xã hội ở Việt Nam đã đạt được nhiều thành tựu: Lực lượng sản xuất phát triển với hàm lượng khoa học ngày càng cao và với quan hệ sản xuất phù hợp theo định hướng xã hội chủ nghĩa; kinh tế tăng trưởng nhanh, chất lượng được cải thiện; quyền làm chủ của nhân dân đối với mọi mặt đời sống xã hội được bảo đảm; Nhà nước pháp quyền xã hội chủ nghĩa của dân, do dân, vì dân ngày càng hoàn thiện theo hướng trong sạch, vững mạnh. Dân chủ được phát huy, kỷ luật, kỷ cương được bảo đảm; văn hóa, giáo dục - đào tạo, khoa học - công nghệ được mở mang, trình độ dân trí phát triển; những thói hư, tật xấu, tệ nạn xã hội được đấu tranh loại bỏ dần; môi trường sinh thái được quan tâm bảo vệ hơn; con người có điều kiện hơn để phát triển về thể chất, trí tuệ, đạo đức, nghề nghiệp, có cuộc sống ngày càng ấm no, hạnh phúc; được cống hiến và hưởng thụ công bằng thành quả của sự phát triển. 
Tuy nhiên, việc thực hiện tiến bộ xã hội vẫn còn những hạn chế, như tình trạng phân hóa giàu nghèo, tỷ lệ thất nghiệp cao, giảm nghèo chưa bền vững, bất bình đẳng trong thu nhập, chênh lệch mức sống ngày càng tăng, không ít giá trị văn hóa, đạo đức bị mai một, xuống cấp.
Quá trình đổi mới, hội nhập quốc tế đi vào chiều sâu, những thành quả của cuộc cách mạng khoa học - công nghệ, nhất là cuộc cách mạng công nghiệp 4.0 đã đặt ra nhiều vấn đề mới cho Việt Nam, phát triển kinh tế, văn hóa, thực hiện tiến bộ và công bằng xã hội, mặt trái của quá trình toàn cầu hóa, kinh tế thị trường.... Do đó, trong thời gian tới,  để giải quyết mối quan hệ giữa tăng trưởng kinh tế với thực hiện tiến bộ và công bằng xã hội ở nước ta thì cần tập trung vào một số giải pháp sau đây.
Một là, tiếp tục nâng cao nhận thức của các cấp ủy đảng, chính quyền về phát triển kinh tế gắn với thực hiện tiến bộ, công bằng xã hội trong điều kiện mới. 
Hai là, tiếp tục thể chế hóa các quan điểm, đường lối của Đảng, đặc biệt là Nghị quyết Đại hội XIII về phát triển kinh tế, văn hóa, thực hiện tiến bộ và công bằng xã hội thành các chính sách, pháp luật của Nhà nước, xử lý hài hòa mối quan hệ giữa phát triển kinh tế, xây dựng Đảng và hệ thống chính trị với phát triển văn hóa; khắc phục tình trạng chạy theo lợi ích kinh tế, không quan tâm đúng mức các giá trị văn hóa.
Ba là, quan tâm, giải quyết tốt những mâu thuẫn trong quá trình phát triển, như: thực hiện tiến bộ và công bằng xã hội trong điều kiện kinh tế phát triển chưa cao, nguồn lực còn hạn chế; tăng trưởng kinh tế với xây dựng văn hóa, thực hiện tiến bộ và công bằng xã hội. 
Bốn là, phát triển và nâng cao chất lượng nguồn nhân lực, nhất là nguồn nhân lực chất lượng cao là một đột phá chiến lược, là yếu tố quyết định đẩy sự phát triển. Quán triệt quan điểm tiến bộ và công bằng xã hội trong thực hiện chủ trương phát triển nguồn nhân lực trong các lĩnh vực quan trọng, đặc biệt là các khu vực còn nhiều khó khăn, miền núi, vùng sâu, vùng xa, các dân tộc thiểu số. 
Năm là, xây dựng, củng cố, phát huy thiết chế xã hội các cấp trong việc gắn kết phát triển kinh tế với thực hiện tiến bộ và công bằng xã hội. Xây dựng, hoàn thiện cơ sở vật chất phục vụ phát triển xã hội, quyền tiếp cận bình đẳng của người dân và phục vụ nhu cầu phát triển kinh tế, xã hội ở địa phương.
Giải quyết đúng đắn mối quan hệ giữa tăng trưởng kinh tế và phát triển văn hóa, thực hiện tiến bộ và công bằng xã hội là điều kiện của sự phát triển nhanh, hiệu quả và bền vững. Theo đó, chúng ta không chờ kinh tế đạt đến trình độ phát triển cao mới phát triển văn hóa, thực hiện tiến bộ và công bằng xã hội, càng không hy sinh văn hóa, hy sinh tiến bộ và công bằng xã hội để chạy theo tăng trưởng kinh tế đơn thuần. Mỗi chính sách phát triển kinh tế đều phải hướng tới phát triển bền vững, hiệu quả, phát triển văn hóa, thực hiện tiến bộ và công bằng xã hội; mỗi chính sách phát triển văn hóa, bảo đảm tiến bộ và công bằng xã hội đều phải góp phần thúc đẩy tăng trưởng kinh tế, dù trực tiếp hay gián tiếp, trước mắt hoặc lâu dài.
Thành phố khai thác có hiệu quả lợi thế của một đô thị đặc biệt, một trung tâm lớn về kinh tế, văn hóa, giáo dục đào tạo, khoa học công nghệ vào phát triển kinh tế tri thức, hình thành những nền tảng của kinh tế tri thức, bao gồm đội ngũ trí thức đông đảo, đô thị thông minh, khu công nghệ cao, trung tâm khởi nghiệp sáng tạo, các tập đoàn lớn về công nghệ, các trung tâm nghiên cứu phát triển, hạ tầng công nghệ thông tin trên nền tảng mạng 5G. Trên cơ sở đó, từng bước chuyển dịch cơ cấu kinh tế theo hướng gia tăng hàm lượng tri thức, tập trung phát triển các ngành công nghệ cao, tham gia vào chuỗi giá trị toàn cầu.
Trong xu thế phát triển khoa học và công nghệ thế giới, cùng với sự tác động của cuộc cách mạng công nghiệp 4.0, sự quan tâm, đầu tư của chính phủ, Nhà nước, Thành phố Hồ Chí Minh không chỉ là trung tâm kinh tế, còn là trung tâm khoa học và công nghệ của cả nước.
Thứ nhất, Thành phố có tiềm lực cơ cở vật chất cho phát triển khoa học và công nghệ. Thành phố Hồ Chí Minh có 3 phòng thí nghiệm trọng điểm, Thành phố có khu công nghệ cao, Khu công nghệ thông tin tập trung, bao gồm: Khu phần mềm Quang Trung, Khu Công nghệ Phần mềm Đại học Quốc gia Thành phố Hồ Chí Minh, Khu E-Tow, Khu Nông nghiệp Công nghệ, các cơ sở ươm tạo công nghệ. Ngoài ra, phòng thí nghiệm được công nhận VILAS trên địa bàn thành phố Hồ Chí Minh là 196 tại thời điểm 31/10/2020. Trong đó chiếm 45% nhà nước và 55% ngoài nhà nước. 
Thứ hai, Thành phố Hồ Chí Minh có nguồn nhân lực chất lượng cao tạo điều kiện phát triển khoa học và công nghệ. Kết quả Tổng điều tra Dân số và Nhà ở năm 2019 cho thấy, dân số TP. Hồ Chí Minh tại thời điểm 01/4/2019 là 8.993.082 người, trong đó 1.508.357 người có trình độ học vấn từ Cao đẳng trở lên, đây là những người được xem là nhân lực KH&CN tiềm năng của Thành phố. Số cán bộ nghiên cứu là 16.064 người, chiếm 75% nhân lực hoạt động R&D. Trong đó, chiếm đến 76% là cán bộ nghiên cứu có trình độ thạc sĩ và tiến sĩ. Các lĩnh vực tập trung nhiều cán bộ nghiên cứu nhất là khoa học kỹ thuật và công nghệ (chiếm 33%), khoa học xã hội (chiếm 30%). Năm 2015 tống số nhân lực nghiên cứu và phát triển khoa học của các tổ chức ở Thành phố Hồ Chí Minh là 17.820 người trong đó cán bộ nghiên cứu là 14.544 người chiếm 82%; nhân viên k thuật 855 người chiếm 5%; nhân viên hỗ trợ là 2.421 chiếm 14%. Nhưng đến năm 2018 con số này tăng lên nhiều với 21.300 người trong đó cán bộ nghiên cứu là 16.064 chiếm 75%, nhân viên k thuật là 1.237 chiếm 6%; nhân viên hỗ trợ là 3.999 người chiếm 19%, qua số liệu ta thấy số nhân lực nghiên cứu và phát triển khoa học và công nghệ của Thành phố có xu hướng tăng, trong đó, nhân viên kĩ thuật và nhân viên hỗ trợ có sự tăng lên, điều này phản ánh nhu cầu tăng hoạt động thực nghiệm khoa học. 
Thứ ba, nguồn lực đầu tư khoa học và công nghệ ở Thành phố Hồ Chí Minh đa dạng. Nguồn tài chính đầu tư cho phát triển, nghiên cứu khoa học và công nghệ ở Thành phố Hồ Chí Minh được cấu thành từ các nguồn: ngân sách nhà nước, đầu tư của doanh nghiệp, vốn của nước ngoài. Tỷ lệ bình quân mức đầu tư của toàn xã hội cho KH&CN/GRDP TP Hồ Chí Mịnh giai đoạn 2015-2020 là 0.96%. Với mục tiêu đến năm 2025 của thành phố Hồ Chí Minh chi đầu tư xã hội cho KH&CN/GRDP đạt bình quân trên 1%. 
Thứ tư, Thành phố Hồ Chí Minh có nhiều chương trình, hoạt động nghiên cứu khoa học và công nghệ. Thành phố có quá trình hoạt động nghiên cứu khoa học phát triển, với những công nghệ mới được tạo ra, sự liên kết giữa các nghiên cứu trên thế giới diễn ra thường xuyên và mang lại hiệu quả ngày càng cao. Thứ năm, Thành phố Hồ Chí Minh thúc đẩy việc kết nối và hình thành mối liên kết hợp tác giữa các Trường, Viện, Tổ chức khoa học và công nghệ với doanh nghiệp. Để triển khai các hoạt động nghiên cứu khoa học, chuyển giao, ứng dụng, thương mại hóa kết quả nghiên cứu, đào tạo và quản lý trong lĩnh vực nhằm hình thành các sản phẩm thuộc nhóm sản phẩm chủ lực, tiềm năng của Thành phố.
Thứ nhất, vai trò của khoa học và công nghệ đối với sự thay đổi số lượng và chất lượng nguồn nhân lực chất lượng cao. Thành phố Hồ Chí Minh còn là nơi tập trung nhiều khu công nghiệp, khu chế xuất, với khu công nghệ cao, khu nông nghiệp công nghệ cao, nên rất cần một nguồn nhân lực chất lượng cao để đáp ứng quá trình phát triển. Hơn nữa, ở Thành phố có nhiều trường Đại học, Cao đẳng, Học viện, Viện nghiên cứu nên số lượng lao động được đào tạo ra mỗi năm gia tăng, nguồn nhân lực nghiên cứu và phát triển trên địa bàn Thành phố luôn chiếm một số lượng lớn của cả nước. 
Nếu năm 2011 nguồn nhân lực nghiên cứu và phát triển của các tổ chức khoa học và công nghệ của cả nước có 115.522, trong đó Thành phố là 17.820 người (chiếm 15%), thì đến năm 2017, nguồn nhân lực nghiên cứu và phát triển của Thành phố tăng lên 21.300 người, trong đó tổng số của cả nước là 147.491 người (chiếm 15%). 
Theo CSDL Scopus, số bài báo của Việt Nam công bố trên các tạp chí KH&CN quốc tế giai đoạn 2014 - 2019 đã tăng gấp ba lần, từ 4.071 bài lên 12.431 bài. Xét về tốc độ tăng trưởng bình quân hằng năm, số công bố quốc tế tăng bình quân khoảng 23%/năm trong giai đoạn 2014 – 2019, đặc biệt tăng mạnh trong 2 năm gần đây, với tốc độ bình quân năm 2019 tăng 40.93% so với năm 2018. Riêng trong năm 2017, mặc dù số lượng công bố có tăng so với những năm trước với 6,602 công bố nhưng tốc độ tăng thấp nhất so với các năm từ 2014 – 2019. Xem xét các tổ chức có nhiều công bố quốc tế tại Việt Nam thì top 10 tổ chức có năng suất công bố khoa học tốt nhất cả nước năm 2019 gồm 6 trường đại học (ĐH) công lập. 2 trường ĐH tư thục và 2 viện nghiên cứu. Trong số 10 tổ chức này. thành phố Hồ Chí Minh (TP HCM) vinh dự góp mặt 4 đại diện tiêu biểu: trường ĐH Tôn Đức Thắng. trường ĐH Công nghệ TP HCM. ĐH Quốc gia TP HCM. Viện Khoa học và Công nghệ tính toán (Hình 3)."""

text2 = """Ở Việt Nam, công nghiệp hóa, hiện đại hóa là một trong những giải pháp quyết định đưa nước ta sớm thoát khỏi tình trạng nghèo nàn lạc hậu, khắc phục nguy cơ tụt hậu về kinh tế, tiến kịp với các nước trong khu vực và trên thế giới, đồng thời cải thiện đời sống của nhân dân, tăng cường tiềm lực quốc phòng - an ninh, củng cố vững chắc độc lập chủ quyền của Tổ quốc. Qua đó giải phóng những tiềm lực to lớn của đất nước và mở ra trước mắt chúng ta những sự thay đổi thần kỳ trong phát triển kinh tế giúp nước ta vươn lên trở thành nước có mức thu nhập trung bình thấp, vượt qua thời kỳ khủng hoảng kinh tế -xã hội, đói nghèo và kém phát triển. Đặc biệt, cuộc cách mạng công nghiệp lần thứ tư được cho là đã bắt đầu từ vài năm gần đây là cuộc cách mạng về sản xuất thông minh dựa trên các thành tựu đột phá trong các lĩnh vực công nghệ thông tin, công nghệ sinh học, công nghệ nano…với nền tảng là các đột phá của công nghệ số. Cách mạng công nghiệp lần thứ tư sẽ mở ra cơ hội phát triển cho công nghiệp hóa, hiện đại hóa ở Việt Nam. 
Chính vì thế, việc nhìn nhận và đánh giá thực trạng của công nghiệp hóa, hiện đại hóa, hội nhập quốc tế hiện nay là một vấn đề hết sức quan trọng đối với Đảng ta. Đảng phải biết cách vận dụng sáng tạo trong trong việc đẩy mạnh công nghiệp hóa, hiện đại hóa để phát triển kinh tế - xã hội  tạo điều kiện tăng cường củng cố an ninh - quốc phòng và là tiền đề cho việc xây dựng một nền kinh tế độc lập, tự chủ, đủ sức tham gia một cách có hiệu quả vào sự phân công và hợp tác quốc tế.
Vì lẽ đó, trong báo cáo lần này nhóm trình bày về những quan điểm và giải pháp của Đảng trong việc góp phần đẩy mạnh công nghiệp hóa, hiện đại hóa, hội nhập quốc tế ở Việt Nam hiện nay, cũng như áp dụng những quan điểm, chủ trương của Đảng và nhà nước để nhìn nhận các vấn đề, thực trạng trong xã hội Việt Nam hiện nay.
Trong đề tài này nhóm sẽ tập trung rõ các vấn đề cốt lõi trong quá trình đẩy mạnh công nghiệp hóa, hiện đại hóa, hội nhập quốc tế ở Việt Nam hiện nay.
Hiểu rõ và vận dụng được quan điểm chỉ đạo của Đảng về công nghiệp hóa, hiện đại hóa và hội nhập quốc tế đất nước.
Đánh giá, phân tích thực trạng và đề xuất các giải pháp cần thiết cho trong quá trình công nghiệp hóa, hiện đại hóa, hội nhập quốc tế ở Việt Nam hiện nay.
Nhằm tập trung vào vấn đề trung tâm của đề tài nên nhóm đã xác định ra phạm vi nghiên cứu là xác định tìm giải pháp cho giải quyết vấn đề hiện tại để tiến hành hành công nghiệp hóa, hiện đại hóa, hội nhập quốc tế phạm vi địa bàn TP.Hồ Chí Minh, trong vòng 10 năm trở lại đây để đề xuất giải pháp thực hiện từ 2021 trở về sau.
Bước vào thời kỳ đổi mới trên cơ sở phân tích khoa học các điều kiện trong nước và quốc tế, Đảng đã nêu ra những quan điễm chĩ đạo quá trình thực hiện công nghiệp hóa, hiện đại hóa đất nước trong điều kiện mới.
Những quan điểm này được hội nghị lần thứ bảy Ban Chấp hành Trung ương khóa VII nêu ra và được phát triên bổ sung qua các Đại hội VIII, IX, X, XI, XII của Đảng. Những quan điễm đó được làm rõ như sau.
Lịch sử công nghiệp hóa trên thế giới đã trải qua hàng trăm năm. Vào giữa thế kỷ XVIII, một số nước phương Tây, mở đầu là nước Anh đã tiến hành cuộc cách mạng công nghiệp, với nội dung chủ yếu là chuyển từ lao động thủ công sang lao động cơ khí. Đây là mốc đánh dấu sự khởi đầu cho tiến trình công nghiệp hóa của thế giới. Tuy vậy, phải đến thế kỷ XIX, khái niệm công nghiệp hóa mới được dùng để thay thế cho khái niệm cách mạng công nghiệp, mặc dù sau cách mạng công nghiệp ở Anh, một thế hệ công nghiệp hóa đã diễn ra ở các nước Tây Âu, Bắc Mỹ và Nhật Bản. Có thể khái quát, công nghiệp hóa là quá trình tạo sự chuyển biến từ nền kinh tế nông nghiệp với nền kinh tế lạc hậu, dựa trên lao động thủ công, năng suất thấp sang nền kinh tế công nghiệp với cơ cấu kinh tế hiện đại, dựa trên lao động sử dụng bằng máy móc, tạo ra năng suất lao động cao. Như vậy, công nghiệp hóa là quá trình biến một nước có nền kinh tế lạc hậu thành nước công nghiệp hiện đại với trình độ công nghệ, kỹ thuật tiên tiến, có năng suất lao động cao trong các ngành kinh tế quốc dân. Hiện đại hóa là quá trình tận dụng mọi khả năng để đạt trình độ công nghệ ngày càng tiên tiến, hiện đại.
Trong điều kiện của Việt Nam, Đảng ta xác định: “Công nghiệp hóa, hiện đại hóa là quá trình chuyển đổi căn bản, toàn diện các hoạt động sản xuất, kinh doanh, dịch vụ và quản lý kinh tế - xã hội từ sử dụng lao động thủ công là chính sang sử dụng một cách phổ biến sức lao động cùng với công nghệ, phương tiện, phương pháp tiên tiến, hiện đại dựa trên sự phát triển của công nghiệp và tiến bộ khoa học - công nghệ, tạo ra năng suất lao động xã hội cao”.
Quan điểm đẩy mạnh công nghiệp hoá, hiện đại hoá (CNH, HĐH) gắn với phát triển kinh tế tri thức (KTTT) đã được Đảng ta chính thức đề cập tại Đại hội X: “Tranh thủ cơ hội thuận lợi do bối cảnh quốc tế tạo ra và tiềm năng, lợi thế của nước ta để rút ngắn quá trình CNH, HĐH đất nước theo định hướng xã hội chủ nghĩa gắn với phát triển KTTT, coi KTTT là yếu tố quan trọng của nền kinh tế và CNH, HĐH. Phát triển mạnh các ngành và sản phẩm kinh tế có giá trị gia tăng cao dựa nhiều vào tri thức; kết hợp việc sử dụng nguồn vốn tri thức của con người Việt Nam với tri thức mới nhất của nhân loại” . Trong Cương lĩnh xây dựng đất nước trong thời kỳ quá độ lên chủ nghĩa xã hội (bổ sung, phát triển năm 2011) đã phát triển và cụ thể hoá thêm một bước quan điểm đẩy mạnh CNH, HĐH gắn với phát triển KTTT: “Phát triển kinh tế là nhiệm vụ trung tâm; thực hiện CNH, HĐH đất nước gắn với phát triển KTTT; …Phát triển khoa học và công nghệ nhằm mục tiêu đẩy mạnh CNH, HĐH đất nước, phát triển KTTT, vươn lên trình độ tiên tiến của thế giới”.
Thực tế lịch sử đã chứng minh, trong thời đại toàn cầu hoá và hội nhập quốc tế, tất cả các nước có xuất phát điểm thấp, muốn vươn lên trình độ tiên tiến của thế giới không có con đường nào khác ngoài việc thực hiện CNH, HĐH gắn với phát triển KTTT.
Đây chính là xu thế khách quan của thời đại toàn cầu hoá; đồng thời cũng là con đường “rút ngắn” của quá trình công nghiệp hoá theo hướng hiện đại nhằm chuyển nền kinh tế nông nghiệp thành nền kinh tế công nghiệp - tri thức - và nền KTTT.
Thực ra, từ những thập kỷ cuối thế kỷ XX cho tới nay, khoa học và công nghệ đã có những bước phát triển kỳ diệu, đặc biệt là sự bùng nổ và sự hội tụ của công nghệ thông tin, công nghệ sinh học, công nghệ nanô, đúng như những tiên đoán của C.Mác và Ph.Ăngghen từ giữa thế kỷ XIX, “tri thức sẽ trở thành lực lượng sản xuất trực tiếp”; “giá trị của lao động cơ bắp trong sản phẩm làm ra sẽ giảm còn cực nhỏ”; “lực lượng sản xuất tinh thần”; “sự xuất hiện công nhân khoa học”... thúc đẩy mạnh mẽ quá trình chuyển đổi từ nền kinh tế công nghiệp sang  nền KTTT.
Thuật ngữ “kinh tế tri thức” (knowledge economy), mặc dù xuất hiện từ khá sớm, nhưng  mãi tới năm 1990 mới chính thức được ghi nhận trong báo cáo của Liên hiệp quốc và từ đó trên sách báo đã xuất hiện những định nghĩa khác nhau về KTTT. Năm 1996, OECD đưa ra định nghĩa: “Nền KTTT là nền kinh tế trong đó sự sản sinh ra, phổ cập và sử dụng tri thức giữ vai trò quyết định nhất đối với sự phát triển kinh tế, tạo ra của cải, nâng cao chất lượng cuộc sống” .
Năm 2000, APEC cũng quan niệm: Nền KTTT là nền kinh tế trong đó sự sản sinh ra, truyền bá và sử dụng tri thức là động lực chủ yếu nhất của sự tăng trưởng, tạo ra của cải, tạo ra việc làm trong tất cả các ngành kinh tế.
UNDP và Ngân hàng thế giới đưa ra định nghĩa: Nền KTTT là nền kinh tế sử dụng có hiệu quả tri thức cho phát triển kinh tế và xã hội, bao gồm cả việc khai thác kho tri thức toàn cầu, cũng như làm chủ và sáng tạo tri thức cho những nhu cầu của riêng mình.
Dù còn có những định nghĩa khác nhau, song có thể nhận diện nền KTTT là nền kinh tế sử dụng có hiệu quả tri thức cho phát triển kinh tế và xã hội; đó là nền kinh tế với những đặc trưng chủ yếu sau.
Khác với nền kinh tế nông nghiệp chủ yếu phụ thuộc vào tự nhiên, chủ thể chính là nông dân cần cù trong sử dụng “cày chìa vôi” để sản xuất; nền kinh tế công nghiệp, chủ thể là công nhân thành thạo trong thao tác các công cụ cơ khí, nền KTTT là nền kinh tế mà chủ thể của nó là công nhân - trí thức sáng tạo trong điều khiển các công cụ tự động hoá, các công cụ truyền thông đa phương tiện, sử dụng có hiệu quả tri thức và công nghệ để phát triển kinh tế, xã hội và bảo vệ môi trường sinh thái.
Hoạt động chủ yếu nhất trong nền KTTT là hoạt động tạo ra, quảng bá và sử dụng hiệu quả tri thức, biến tri thức thành giá trị. Tri thức đã trở thành nguồn lực hàng đầu, mà giá trị của nó chiếm tỷ trọng cao, thậm chí là tuyệt đối trong sản phẩm. Với nền KTTT, nhân tài trí thức được coi trọng, quyền  sở hữu trí tuệ trở thành yếu tố quan trọng; tri thức trở thành hình thức cơ bản nhất của vốn, quan trọng hơn cả tài nguyên, sức lao động; lực lượng sản xuất từ dựa vào tài nguyên thiên nhiên chuyển dần sang dựa chủ yếu vào năng lực trí tuệ của con người.
Nền KTTT phát huy tối đa  năng lực sáng tạo của con người trong vận dụng và sáng tạo ra tri thức mới, thúc đẩy tăng trưởng kinh tế, phát triển con người và xã hội một cách nhanh chóng. Bởi vậy, vòng đời của một sản phẩm, một công nghệ từ lúc nảy sinh, phát triển, chín muồi đến tiêu vong ngày càng rút ngắn; trước đây vòng đời công nghệ tính bằng nhiều thập niên, ngày nay tính bằng năm, thậm chí tính bằng tháng. Tốc độ đổi mới công nghệ rất nhanh chóng.
Sau 25 năm vượt qua các thử thách và khó khăn to lớn, tiến hành công cuộc đổi mới trên mọi mặt, đến nay Việt Nam có đủ cơ sở để khẳng định rằng, đổi mới là sự lựa chọn đúng đắn, đã đặt Việt Nam vào quỹ đạo phát triển mới về chất, phù hợp với xu thế của thời đại và ý nguyện của nhân dân.
Ngày nay, công cuộc đổi mới đã chuyển sang giai đoạn phát triển mới: đổi mới và hội nhập; đổi mới để giữ vững ổn định chính trị một cách tích cực cho phát triển bền vững; đổi mới hệ thống chính trị phải tương thích với đổi mới kinh tế nhằm xây dựng và hoàn thiện nền dân chủ xã hội chủ nghĩa bảo đảm toàn bộ quyền lực thuộc về nhân dân và đổi mới tạo nền tảng để đến năm 2020 nước ta cơ bản trở thành nước công nghiệp theo hướng hiện đại.
Đổi mới cũng là quá trình vừa làm vừa tìm tòi, rút kinh nghiệm; kinh nghiệm của chính bản thân chúng ta và kinh nghiệm của thế giới. Đó là quá trình tiếp thu có chọn lọc hết sức nghiêm túc và sáng tạo.
Một phần tư thế kỷ qua, đổi mới không chỉ là một ý chí, mong muốn, một khẩu hiệu tuyên truyền mà là quyết tâm chính trị - hành động lan tỏa, thấm sâu trong mọi tế bào của đời sống xã hội, làm thay đổi nếp nghĩ, cách làm và gây dựng cơ nghiệp.
Đổi mới đã làm thay đổi căn bản và toàn diện trên tất cả các lĩnh vực từ kinh tế, chính trị, văn hóa, xã hội đến an ninh, quốc phòng và đối ngoại: Kinh tế tăng trưởng nhanh, đạt tốc độ bình quân 7,2%/năm. Năm 2010, tổng sản phẩm trong nước bình quân đầu người đạt 1.200 USD. Cơ cấu kinh tế chuyển dịch theo hướng tích cực. Thể chế kinh tế thị trường định hướng xã hội chủ nghĩa tiếp tục được hoàn thiện. Các lĩnh vực văn hoá, xã hội đạt thành tựu quan trọng trên nhiều mặt. Đời sống vật chất và tinh thần của nhân dân được cải thiện rõ rệt; dân chủ trong xã hội tiếp tục được mở rộng. Chính trị - xã hội ổn định; quốc phòng, an ninh được giữ vững. Công tác đối ngoại, hội nhập quốc tế được triển khai sâu rộng và hiệu quả góp phần tạo môi trường hòa bình, ổn định và tăng thêm nguồn lực cho phát triển đất nước.
Diện mạo của đất nước có nhiều thay đổi. Thế và lực của đất nước vững mạnh thêm nhiều; vị thế của Việt Nam trên trường quốc tế được nâng lên, tạo ra những tiền đề quan trọng để đẩy nhanh công nghiệp hoá, hiện đại hóa đất nước và nâng cao chất lượng cuộc sống của nhân dân.
Những thành tựu to lớn và có ý nghĩa lịch sử ấy đã nâng cao tầm vóc và vị thế của đất nước ta, ngày càng được bạn bè quốc tế đánh giá cao và nhiệt tình ủng hộ. Tuy vậy, kinh tế phát triển chưa bền vững; chất lượng, hiệu quả, sức cạnh tranh thấp, chưa tương xứng với tiềm năng, cơ hội và yêu cầu phát triển của đất nước. Nền kinh tế chủ yếu vẫn dựa vào tài nguyên và lao động, giá trị do tri thức tạo ra không đáng kể; chưa khơi dậy và phát huy tối đa năng lực sáng tạo của con người; nền kinh tế còn chứa đựng nhiều yếu tố phát triển không bền vững, có nguy cơ tụt hậu xa hơn so với các nước khác. Về cơ bản nước ta vẫn là nước chưa phát triển.
Là một nước nông nghiệp đi lên, trong bối cảnh toàn cầu hoá và hội nhập quốc tế để phát triển, nền KTTT cần tiến hành đồng thời và lồng ghép của hai quá trình: quá trình chuyển từ kinh tế nông nghiệp lên kinh tế công nghiệp và quá trình chuyển từ kinh tế công nghiệp lên KTTT, trong đó nhất thiết phải bắc được những “nhịp cầu trung gian” để thực hiện những bước chuyển quá độ. Đây là cả một sự nghiệp vô cùng khó khăn, phức tạp. Trong khi đối với các nước đi trước, để thực hiện bước chuyển này chỉ là hai quá trình kế tiếp nhau, còn đối với nước ta để thực hiện bước phát triển “rút ngắn” đòi hỏi phải kết hợp các bước đi tuần tự với các bước phát triển nhảy vọt. Một mặt, tận dụng lao động, cơ sở vật chất hiện có, sử dụng tri thức mới, công nghệ mới để phát triển sản xuất, tạo việc làm, tăng thu nhập. Mặt khác, đi thẳng vào hiện đại ở những khâu, những lĩnh vực, những ngành có lợi thế, phát triển các ngành, các vùng kinh tế mũi nhọn tạo thành đầu tầu có sức kéo lôi mạnh toàn bộ nền kinh tế đi lên.
Bởi vậy, việc chuyển nền kinh tế theo hướng phát triển chủ yếu dựa vào tri thức trở thành yêu cầu cấp thiết, không thể trì hoãn. Bỏ lỡ thời cơ lớn này, Việt Nam sẽ tụt hậu xa hơn và đó là hiểm họa của dân tộc. Báo cáo chính trị tại Đại Hội X của Đảng đã chỉ ra: “Tranh thủ cơ hội thuận lợi do bối cảnh quốc tế tạo ra và tiềm năng, lợi thế của nước ta để rút ngắn quá trình CNH, HĐH đất nước theo định hướng XHCN gắn với phát triển KTTT, coi KTTT là yếu tố quan trọng của nền kinh tế và CNH, HĐH. Phát triển mạnh các ngành và sản phẩm kinh tế có giá trị gia tăng cao dựa nhiều vào tri thức; kết hợp việc sử dụng nguồn vốn tri thức của con người Việt Nam với tri thức mới nhất của nhân loại” .  Đại hội XI, trên tinh thần tiếp tục nâng cao năng lực lãnh đạo và sức chiến đấu của Đảng, phát huy sức mạnh toàn dân tộc, đẩy mạnh toàn diện công cuộc đổi mới, tạo nền tảng để đến năm 2020 nước ta cơ bản trở thành nước công nghiệp theo hướng hiện đại, đã khẳng định: Để xây dựng thành công chủ nghĩa xã hội và bảo vệ vững chắc Tổ quốc xã hội chủ nghĩa, phải tiếp tục đổi mới toàn diện, mạnh mẽ với chất lượng và hiệu quả cao hơn; đẩy mạnh CNH, HĐH gắn với phát triển KTTT; phát triển kinh tế thị trường gắn với giải quyết hài hoà các vấn đề xã hội, môi trường...
Trong vài ba thập kỷ đầu của thế kỷ XXI, để thực hiện có kết quả bước phát triển “rút ngắn” sớm xác lập nền KTTT Việt Nam, cần thiết phải tiến hành đồng bộ nhiều giải pháp, trong đó cần tập trung toàn lực vào việc thực hiện những giải pháp mang tính đột phá sau.
Một là, tiếp tục cải cách mạnh mẽ, có hiệu quả nền giáo dục và đào tạo theo hướng nâng cao dân trí, đào tạo nhân lực, bồi dưỡng và phát huy nhân tài.
Cải cách triệt để nền giáo dục được coi là “đột phá khẩu”, bởi nó hướng tới việc đào tạo, bồi dưỡng và phát huy nguồn nhân lực có chất lượng cao cho quá trình tái cơ cấu lại nền kinh tế, tăng lợi thế cạnh tranh, bảo đảm phát triển nhanh, hiệu quả, bền vững kinh tế - xã hội, đẩy mạnh CNH, HĐH gắn với phát triển KTTT. Dự thảo Chiến lược phát triển kinh tế - xã hội 2011 - 2020 khẳng định: “Phát triển và nâng cao chất lượng nguồn nhân lực, nhất là nguồn nhân lực chất lượng cao là một đột phá chiến lược, là yếu tố quyết định quá trình cơ cấu lại nền kinh tế, chuyển đổi mô hình tăng trưởng và là lợi thế cạnh tranh dài hạn, bảo đảm kinh tế - xã hội phát triển nhanh, hiệu quả, bền vững... Thực hiện các chương trình, đề án đào tạo nhân lực chất lượng cao đối với các ngành, lĩnh vực chủ yếu, mũi nhọn. Chú trọng phát hiện, bồi dưỡng, phát huy nhân tài; đào tạo nhân lực cho phát triển KTTT”.
Hai là, phát triển mạnh khoa học, công nghệ, chú trọng vào việc việc ứng dụng, sáng tạo công nghệ cao làm động lực đẩy nhanh quá trình CNH, HĐH, phát triển KTTT.
Đã từ lâu, chúng ta quan tâm tới việc phát triển khoa học và công nghệ, song trong lĩnh vực công nghệ chúng ta chưa tập trung vào việc vận dụng và sáng tạo, phát triển công nghệ cao.
Thực tế cho thấy, việc thiếu cơ chế chính sách trong việc tập trung cho công nghệ cao không tạo động lực cho phát triển; còn công nghệ lạc hậu sẽ cản trở, kìm hãm sự phát triển của CNH, HĐH.
Đã đến lúc cần phải “tuyên chiến” với công nghệ lạc hậu, chỉ nhập khẩu, ứng dụng và đầu tư sáng tạo công nghệ cao trên cơ sở sử dụng có hiệu quả những công nghệ hiện có. Là nước đi sau, Việt Nam có thể bỏ qua các thế hệ công nghệ trung gian để đi ngay vào các công nghệ tiên tiến, công nghệ cao, không lệ thuộc vào cơ sở hạ tầng đã có. Trong các lĩnh vực công nghiệp dựa vào tri thức, nhất là công nghệ thông tin, Việt Nam có thể chọn một số lĩnh vực để bứt phá lên trước.
Nhiệm vụ trung tâm là sử dụng tri thức mới, công nghệ mới của thời đại kết hợp với sáng tạo tri thức mới, công nghệ mới để nhanh chóng chuyển dịch cơ cấu kinh tế theo hướng tăng nhanh các ngành công nghiệp và dịch vụ hàm lượng tri thức cao, giá trị gia tăng cao. Đây chính là yêu cầu và nội dung của công cuộc CNH, HĐH gắn với phát triển KTTT trong thời đại toàn cầu hoá với một trật tự “thế giới phẳng”.
Báo cáo chính trị của Ban Chấp hành Trung ương Đảng khóa X tại Đại hội đại biểu toàn quốc lần thứ XI, khẳng định: “Phát triển mạnh khoa học, công nghệ làm động lực đẩy nhanh quá trình CNH, HĐH, phát triển KTTT; góp phần tăng nhanh năng suất, chất lượng, hiệu quả, sức cạnh tranh của nền kinh tế, sự phát triển nhanh, bền vững của đất nước; nâng tỉ lệ đóng góp của yếu tố năng suất tổng hợp vào tăng trưởng. Thực hiện đồng bộ ba nhiệm vụ chủ yếu: nâng cao năng lực khoa học, công nghệ; đổi mới cơ chế quản lý; đẩy mạnh nghiên cứu ứng dụng” .
Ba là, tiếp tục đổi mới, phát huy vai trò quản lý, điều hành ở tầm vĩ mô của Nhà nước trong cải cách giáo dục - đào tạo, phát triển khoa học, công nghệ và trong quản lý và phát triển kinh tế thị trường - xã hội.
Vai trò quản lý của Nhà nước trong điều kiện mới phải được xem như là một quá trình tạo điều kiện cho việc thúc đẩy phát triển chứ không đơn thuần là sự thực thi những quyết sách không sát với thực tiễn.
Từ Hội Nghị Trung ương 10/ khoá IX, với chủ trương gắn phát triển kinh tế là nhiệm vụ trung tâm với xây dựng Đảng là khâu then chốt và xây dựng nền văn hoá tiên tiến đậm đà bản sắc dân tộc làm nền tảng tinh thần cho phát triển nhanh, hiệu quả và bền vững ở nước ta, Đảng ta cũng đồng thời chỉ ra và khẳng định ba loại động lực chính cho phát triển: phát triển kinh tế thị trường định hướng XHCN; nâng cao năng lực và sức chiến đấu của Đảng và xây dựng; phát triển nền văn hoá tiên tiến đậm đà bản sắc dân tộc. Dù trên lĩnh vực nào thì yêu cầu sáng tạo và đổi mới cũng luôn là năng lực nội sinh của quá trình phát triển kinh tế - xã hội. Bởi vậy, tiếp tục đổi mới cũng đồng nghĩa với tiếp tục đẩy mạnh CNH, HĐH gắn với phát triển KTTT tạo nên những điều kiện và động lực to lớn cho phát triển nhanh, hiệu quả và bền vững.
Trong tổng thể những nội dung đổi mới toàn diện, cần tập trung trọng tâm vào việc nhận thức lại và thực hiện đúng chức năng, vai trò của nhà nước đối với phát triển kinh tế, từ điều khiển, chỉ huy sang là “kiến trúc sư” trong cải cách giáo dục - đào tạo, trong phát triển khoa học, công nghệ, trong việc tạo môi trường thuận lợi và động viên mọi nguồn lực để thực hiện bước chuyển đổi từ nền kinh tế công nghiệp sang nền KTTT. Hơn nữa, cần tăng cường tính minh bạch của chính phủ; phòng chống tham nhũng có hiệu quả, tạo điều kiện cho nhân dân tham gia tích cực, sáng tạo trong việc hoạch định và thực thi chính sách. Trong điều kiện hiện nay, với sự biến đổi nhanh chóng, khó lường của tình hình thế giới; sự phát triển nhanh chóng của khoa học và công nghệ, để phát huy vai trò tạo điều kiện của nhà nước trong đổi mới và phát triển đòi hỏi công tác chỉ đạo, quản lý, điều hành của Nhà nước phải hết sức nhạy bén, kiên quyết, sáng tạo, bám sát thực tiễn đất nước; nâng cao chất lượng hoạt động dự báo, kịp thời đề ra các giải pháp phù hợp với tình hình mới. Trong điều kiện hiện nay, với sự biến đổi nhanh chóng, khó lường của tình hình thế giới, trong đổi mới và quản lý, nâng cao chất lượng hoạt động dự báo.
Quá trình công nghiệp hóa, hiện đại hóa ở nước ta đã thấy rõ và đang trong quá trình phát triển mạnh mẽ. Không những ở thành thị mà cả ở nông thôn.
Song chúng ta cũng đã thấy rõ vấn đề môi trường tự nhiên đang bị ô nhiễm nặng, gây tác động xấu đến sản xuất và đời sống. Những khu công nghiệp, cụm điểm công nghiệp ngày càng nhiều, những khu đô thị mới hiện đại mọc lên, và sản xuất hàng hóa, chế biến nông sản, thực phẩm ở nông thôn ngày càng trở nên sôi động. Do đó, việc giải quyết vấn đề môi trường thông qua các chính sách phát triển kinh tế đi đôi với bảo vệ môi trường đang là vấn đề cấp thiết được Nhà nước đặt ra hiện nay.
Ngày 9/6/2014, Thủ tướng Chính phủ đã ký quyết định số 879/QĐ-TTg phê duyệt chiến lược phát triển công nghiệp Việt Nam đến năm 2025, cụm điểm công nghiệp ngày càng nhiều, tầm nhìn đến 2035.
Trong đó liên quan đến môi trường thì quan điểm phát triển đã nêu rõ: phát triển công nghiệp trên cơ sở tăng trưởng xanh, phát triển bền vững bảo vệ môi trường.
Cụm từ “Phát triển bền vững” đã và đang trở thành tuyên ngôn và chiến lược hành động của nhiều quốc gia trên thế giới. Kể từ Hội nghị Stockhom năm 1987 đến Hội nghị Rio de Janeiro năm 1992 và Johanesbour năm 2002 thì trên 200 quốc gia tham gia đã xác định vấn đề môi trường là yếu tố quyết định cho phát triển bền vững. Ngày nay, bên cạnh những thành tựu kinh tế thì thế giới đang phải đối mặt với nhiều thách thức về đói nghèo, thiên tai, thiếu năng lượng và nguyên liệu, bệnh tật và suy thoái môi trường. Các nước ngày càng nhận thức rõ hơn các nguy cơ tiềm ẩn đe dọa đến sự phát triển bền vững, hệ quả của quá trình công nghiệp hóa thiếu cân nhắc về môi trường. Các chính phát triển bền vững sẽ giúp cho các ngành kinh tế có sự cân nhắc sử dụng các nguồn lực sao cho tiết kiệm và hiệu quả nhất. Các chính sách phát triển bền vững là công cụ điều chỉnh hướng tới những tư duy cân bằng hơn trong cách thức giải quyết và đáp ứng các mục tiêu kinh tế - xã hội và môi trường, nhằm tạo ra những tiền đề phát triển trong tương lai.
Việt Nam là một trong những quốc gia sớm ký tuyên bố chung về thực hiện phát triển bền vững từ năm 1992. Cuối năm 1993, luật Bảo vệ môi trường được ban hành cùng với sự tích cực chuẩn bị Chương trình nghị sự 21 của Việt Nam đã thể hiện quyết tâm đối với các cam kết quốc tế. Những nỗ lực đó của Việt nam được cộng đồng quốc tế đánh giá cao. Tuy nhiên, đó chỉ là những bước khởi đầu của phát triển bền vững, vì Việt nam vẫn đang là nước đang phát triển có thu nhập trung bình thấp, nên nỗi lo toàn về kinh tế đang bao trùm, chi phối các lĩnh vực đời sinh – xã hội và đặc biệt là sản xuất công nghiệp, đặc biệt là sản xuất công nghiệp."""


time: 525 ms (started: 2021-10-09 03:29:31 +00:00)


Let's import a few more libraries for this purpose.


In [5]:
import re                                               # library for regular expression operations
import string                                           # for string operations
from nlp_utils import *                                 # library for standardlize VNM, download from https://gist.github.com/nguyenvanhieuvn/72ccf3ddf7d179b281fdae6c0b84942b
from underthesea import word_tokenize, sent_tokenize    # library for VNM word tokenization

ModuleNotFoundError: ignored

time: 26.6 ms (started: 2021-10-09 03:29:34 +00:00)


### **Number replacement**

Tìm các pattern tương ứng các số và dùng `sub()` method trong thư viện re thay thế bằng một từ như "num".


In [ ]:
print('\033[92m' + text)
print('\033[94m')

# remove date time dd/mm/yy,... ?
# remove currency $100 ?
# remove period 1-2 ?

# remove simple number may be following space or "(" like "(2006)"
rm_num_text = re.sub(r'[\s(][\d]+', ' num', text)

print(rm_num_text)

Hãy thử nhìn lại tình hình của chúng ta. Ngoài những việc đã làm và bước đầu làm được kể ở phần đầu, đã có những cố gắng trong nhiều nội dung khác nhau về xử lý ngôn ngữ và tiếng Việt. Trong các bài giảng về Trí tuệ Nhân tạo, về Lý thuyết Nhận dạng, về Xử lý Tín hiệu, về Khai phá Dữ liệu ở nhiều đại học, các nội dung và kỹ thuật xử lý ngôn ngữ đã ít nhiều được đề cập.Về xử lý tiếng nói và tiếng Việt, theo chúng tôi biết, hai tập thể làm nghiên cứu đã có những kết quả gần đây là Viện Công nghệ Thông tin và Trung tâm nghiên cứu quốc tế Thông tin đa phương tiện, truyền thông và ứng dụng (MICA) – Đại học Bách khoa Hà nội; một số kết quả ở một số trường Đại học là những đề tài tiến sĩ, thạc sĩ, mang tính chất tìm hiểu, chưa hệ thống và định hướng rõ ràng. Nghiên cứu về xử lý ngôn ngữ (văn bản) đã được theo đuổi bởi một số tập thể từ khá lâu (Đại học Bách khoa và Đại học Khoa học Tự nhiên thành phố Hồ Chí Minh, Đại học Bách Khoa Đà Nẵng, Đại học Bách khoa và Đại học Khoa học Tự nhiên Hà Nội,

### **Normalize VNM Unicode**

Chuẩn hóa Unicode tiếng Việt


Hiện nay, có 2 loại mã Unicode được sử dụng phổ biến, Unicode tổ hợp và Unicode dựng sẵn. Điều đó dẫn tới vấn đề dưới đây, do đó ta phải đưa về 1 chuẩn Unicode dựng sẵn.



In [ ]:
print('hiếu' == 'hiếu')
print('hiếu' == 'hiếu')
print('hiếu' == convert_unicode('hiếu'))

True
False
True
time: 10.7 ms (started: 2021-10-09 02:34:03 +00:00)


In [ ]:
print('\033[92m' + rm_num_text)
print('\033[94m')

# Standard VNM unicode
std_uni_text = convert_unicode(rm_num_text)

print(std_uni_text)

Hãy thử nhìn lại tình hình của chúng ta. Ngoài những việc đã làm và bước đầu làm được kể ở phần đầu, đã có những cố gắng trong nhiều nội dung khác nhau về xử lý ngôn ngữ và tiếng Việt. Trong các bài giảng về Trí tuệ Nhân tạo, về Lý thuyết Nhận dạng, về Xử lý Tín hiệu, về Khai phá Dữ liệu ở nhiều đại học, các nội dung và kỹ thuật xử lý ngôn ngữ đã ít nhiều được đề cập.Về xử lý tiếng nói và tiếng Việt, theo chúng tôi biết, hai tập thể làm nghiên cứu đã có những kết quả gần đây là Viện Công nghệ Thông tin và Trung tâm nghiên cứu quốc tế Thông tin đa phương tiện, truyền thông và ứng dụng (MICA) – Đại học Bách khoa Hà nội; một số kết quả ở một số trường Đại học là những đề tài tiến sĩ, thạc sĩ, mang tính chất tìm hiểu, chưa hệ thống và định hướng rõ ràng. Nghiên cứu về xử lý ngôn ngữ (văn bản) đã được theo đuổi bởi một số tập thể từ khá lâu (Đại học Bách khoa và Đại học Khoa học Tự nhiên thành phố Hồ Chí Minh, Đại học Bách Khoa Đà Nẵng, Đại học Bách khoa và Đại học Khoa học Tự nhiên Hà Nội,

### **Normalize VNM accent marks placing**

Chuấn hóa kiểu gõ dấu thanh tiếng Việt, VD: hòa -> hoà


In [ ]:
print(chuan_hoa_dau_cau_tieng_viet("anh Híếu, đang làm gì vâỵ."))

anh Hiếu, đang làm gì vậy.
time: 4.06 ms (started: 2021-10-09 02:34:03 +00:00)


In [ ]:
print('\033[92m' + std_uni_text)
print('\033[94m')

# Standard VNM unicode
std_marks_text = chuan_hoa_dau_cau_tieng_viet(std_uni_text)

print(std_marks_text)

Hãy thử nhìn lại tình hình của chúng ta. Ngoài những việc đã làm và bước đầu làm được kể ở phần đầu, đã có những cố gắng trong nhiều nội dung khác nhau về xử lý ngôn ngữ và tiếng Việt. Trong các bài giảng về Trí tuệ Nhân tạo, về Lý thuyết Nhận dạng, về Xử lý Tín hiệu, về Khai phá Dữ liệu ở nhiều đại học, các nội dung và kỹ thuật xử lý ngôn ngữ đã ít nhiều được đề cập.Về xử lý tiếng nói và tiếng Việt, theo chúng tôi biết, hai tập thể làm nghiên cứu đã có những kết quả gần đây là Viện Công nghệ Thông tin và Trung tâm nghiên cứu quốc tế Thông tin đa phương tiện, truyền thông và ứng dụng (MICA) – Đại học Bách khoa Hà nội; một số kết quả ở một số trường Đại học là những đề tài tiến sĩ, thạc sĩ, mang tính chất tìm hiểu, chưa hệ thống và định hướng rõ ràng. Nghiên cứu về xử lý ngôn ngữ (văn bản) đã được theo đuổi bởi một số tập thể từ khá lâu (Đại học Bách khoa và Đại học Khoa học Tự nhiên thành phố Hồ Chí Minh, Đại học Bách Khoa Đà Nẵng, Đại học Bách khoa và Đại học Khoa học Tự nhiên Hà Nội,

### **Tokenization**

Word Segmentation - Tách từ tiếng Việt

In [ ]:
print('\033[92m' + std_marks_text)
print('\033[94m')

# tokenize text
tokens = word_tokenize(std_marks_text)

print('Tokenized string:')
print(tokens[:100])

print("\nCount number of tokens: ", len(tokens))

Hãy thử nhìn lại tình hình của chúng ta. Ngoài những việc đã làm và bước đầu làm được kể ở phần đầu, đã có những cố gắng trong nhiều nội dung khác nhau về xử lý ngôn ngữ và tiếng Việt. Trong các bài giảng về Trí tuệ Nhân tạo, về Lý thuyết Nhận dạng, về Xử lý Tín hiệu, về Khai phá Dữ liệu ở nhiều đại học, các nội dung và kỹ thuật xử lý ngôn ngữ đã ít nhiều được đề cập.Về xử lý tiếng nói và tiếng Việt, theo chúng tôi biết, hai tập thể làm nghiên cứu đã có những kết quả gần đây là Viện Công nghệ Thông tin và Trung tâm nghiên cứu quốc tế Thông tin đa phương tiện, truyền thông và ứng dụng (MICA) – Đại học Bách khoa Hà nội; một số kết quả ở một số trường Đại học là những đề tài tiến sĩ, thạc sĩ, mang tính chất tìm hiểu, chưa hệ thống và định hướng rõ ràng. Nghiên cứu về xử lý ngôn ngữ (văn bản) đã được theo đuổi bởi một số tập thể từ khá lâu (Đại học Bách khoa và Đại học Khoa học Tự nhiên thành phố Hồ Chí Minh, Đại học Bách Khoa Đà Nẵng, Đại học Bách khoa và Đại học Khoa học Tự nhiên Hà Nội,

###**Sentence segmentation**

Ngoài kỹ thuật Tokenization (Word Segmentation), ta còn có thể phân 1 đoạn văn theo mức độ câu sentence-by-sentence.

In [ ]:
print('\033[92m' + std_marks_text)
print('\033[94m')

# tokenize text by sentences
sentences = sent_tokenize(std_marks_text)

print('Tokenized string:')
print(sentences)

print("Count number of sentences: ", len(sentences))

Hãy thử nhìn lại tình hình của chúng ta. Ngoài những việc đã làm và bước đầu làm được kể ở phần đầu, đã có những cố gắng trong nhiều nội dung khác nhau về xử lý ngôn ngữ và tiếng Việt. Trong các bài giảng về Trí tuệ Nhân tạo, về Lý thuyết Nhận dạng, về Xử lý Tín hiệu, về Khai phá Dữ liệu ở nhiều đại học, các nội dung và kỹ thuật xử lý ngôn ngữ đã ít nhiều được đề cập.Về xử lý tiếng nói và tiếng Việt, theo chúng tôi biết, hai tập thể làm nghiên cứu đã có những kết quả gần đây là Viện Công nghệ Thông tin và Trung tâm nghiên cứu quốc tế Thông tin đa phương tiện, truyền thông và ứng dụng (MICA) – Đại học Bách khoa Hà nội; một số kết quả ở một số trường Đại học là những đề tài tiến sĩ, thạc sĩ, mang tính chất tìm hiểu, chưa hệ thống và định hướng rõ ràng. Nghiên cứu về xử lý ngôn ngữ (văn bản) đã được theo đuổi bởi một số tập thể từ khá lâu (Đại học Bách khoa và Đại học Khoa học Tự nhiên thành phố Hồ Chí Minh, Đại học Bách Khoa Đà Nẵng, Đại học Bách khoa và Đại học Khoa học Tự nhiên Hà Nội,

### **Lowercasing & Remove stop words and punctuations**

Bước tiếp theo là lowercasing các từ để phù hợp với tập stopword là những từ thường. Sau đó loại bỏ các stopword và punctuation. Ta có thể xem danh sách các stopword và punctuation do NLTK cung cấp bên dưới. Danh sách các stopword này có thể gồm những từ quan trọng đối với một vài văn bản, vì vậy tùy trường hợp, chúng tối sẽ xem xét thêm hoặc xóa bớt danh sách.

In [ ]:
# Import the Vietnamese stopwords file, download from: https://github.com/stopwords/vietnamese-stopwords 
f = open('vietnamese-stopwords.txt', 'r')
vnm_stopwords = f.read().splitlines()
f.close()

print('Stop words')
print(vnm_stopwords)

# Get punctuations sring from NLTK
punctuations = """!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~…“”–"""
print('\nPunctuation')
print(punctuations)

Stop words
['a lô', 'a ha', 'ai', 'ai ai', 'ai nấy', 'ai đó', 'alô', 'amen', 'anh', 'anh ấy', 'ba', 'ba ba', 'ba bản', 'ba cùng', 'ba họ', 'ba ngày', 'ba ngôi', 'ba tăng', 'bao giờ', 'bao lâu', 'bao nhiêu', 'bao nả', 'bay biến', 'biết', 'biết bao', 'biết bao nhiêu', 'biết chắc', 'biết chừng nào', 'biết mình', 'biết mấy', 'biết thế', 'biết trước', 'biết việc', 'biết đâu', 'biết đâu chừng', 'biết đâu đấy', 'biết được', 'buổi', 'buổi làm', 'buổi mới', 'buổi ngày', 'buổi sớm', 'bà', 'bà ấy', 'bài', 'bài bác', 'bài bỏ', 'bài cái', 'bác', 'bán', 'bán cấp', 'bán dạ', 'bán thế', 'bây bẩy', 'bây chừ', 'bây giờ', 'bây nhiêu', 'bèn', 'béng', 'bên', 'bên bị', 'bên có', 'bên cạnh', 'bông', 'bước', 'bước khỏi', 'bước tới', 'bước đi', 'bạn', 'bản', 'bản bộ', 'bản riêng', 'bản thân', 'bản ý', 'bất chợt', 'bất cứ', 'bất giác', 'bất kì', 'bất kể', 'bất kỳ', 'bất luận', 'bất ngờ', 'bất nhược', 'bất quá', 'bất quá chỉ', 'bất thình lình', 'bất tử', 'bất đồ', 'bấy', 'bấy chầy', 'bấy chừ', 'bấy giờ', 'bấy lâ

Time to clean up our tokenized text!

In [ ]:
print('\033[92m')
print(tokens)
print('\033[94m')

tokens_clean = []

for word in tokens:                         # Go through every word in your tokens list
    # Lowercasing
    word = word.lower()
    if (word not in vnm_stopwords and       # remove stopwords
        word not in punctuations):          # remove punctuation
        tokens_clean.append(word)

print('After removed stop words and punctuation:')
print(tokens_clean[:100])

print("\nCount number of tokens: ", len(tokens_clean))


['Hãy', 'thử', 'nhìn', 'lại', 'tình hình', 'của', 'chúng ta', '.', 'Ngoài', 'những', 'việc', 'đã', 'làm', 'và', 'bước đầu', 'làm', 'được', 'kể', 'ở', 'phần', 'đầu', ',', 'đã', 'có', 'những', 'cố gắng', 'trong', 'nhiều', 'nội dung', 'khác', 'nhau', 'về', 'xử lý', 'ngôn ngữ', 'và', 'tiếng', 'Việt', '.', 'Trong', 'các', 'bài giảng', 'về', 'Trí tuệ', 'Nhân tạo', ',', 'về', 'Lý thuyết', 'Nhận dạng', ',', 'về', 'Xử lý', 'Tín hiệu', ',', 'về', 'Khai phá', 'Dữ liệu', 'ở', 'nhiều', 'đại học', ',', 'các', 'nội dung', 'và', 'kỹ thuật', 'xử lý', 'ngôn ngữ', 'đã', 'ít nhiều', 'được', 'đề cập', '.', 'Về', 'xử lý', 'tiếng nói', 'và', 'tiếng', 'Việt', ',', 'theo', 'chúng tôi', 'biết', ',', 'hai', 'tập thể', 'làm', 'nghiên cứu', 'đã', 'có', 'những', 'kết quả', 'gần', 'đây', 'là', 'Viện Công nghệ', 'Thông tin', 'và', 'Trung tâm', 'nghiên cứu', 'quốc tế', 'Thông tin', 'đa phương tiện', ',', 'truyền thông', 'và', 'ứng dụng', '(', 'MICA', ')', '–', 'Đại học', 'Bách khoa', 'Hà nội', ';', 'một số', 'kết quả

# **Conclusion**

That's it! Done pre-processing cho văn bản tiếng Việt, yahoo.
Từ 1 văn bản gồm có 7 câu, 523 từ và 2410 ký tự, tokenisation thành list có **469** tokens, và sau khi removal stopwords, punctuations chỉ còn **212** tokens!